In [ ]:
! pip install krwordrank

In [ ]:
! pip install JPype1>=0.7.0

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import TimeoutException

import pandas as pd
import time

In [117]:
import os
import re
from ast import literal_eval
from konlpy.tag import Okt
from collections import Counter
from krwordrank.word import KRWordRank

In [3]:
# 브라우저 열기
driver = webdriver.Chrome()

In [141]:
# 데이터 불러오기
musinsa = pd.read_csv('통합_무신사_데이터.csv')
hwahae = pd.read_csv('통합_화해_데이터.csv')
oliveyoung = pd.read_csv('통합_올리브영_데이터.csv')
cosme = pd.read_csv('통합_코스메_데이터.csv', encoding='EUC-KR')

# 올리브영 크롤링 및 원본 데이터 생성

## 1. 스킨케어 랭킹

In [ ]:
# 1. 스킨케어 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=8&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EB%9E%AD%ED%82%B9BEST%EC%83%81%ED%92%88%EB%B8%8C%EB%9E%9C%EB%93%9C_%EC%9D%B8%EA%B8%B0%EC%83%81%ED%92%88%EB%8D%94%EB%B3%B4%EA%B8%B0")
time.sleep(3)  # 페이지 로드 대기

In [ ]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

In [ ]:
# 4-1. 데이터 텍스트 변환
title_list = [t.text for t in titles]
print(len(title_list))
print(title_list[:5])

product_list = [p.text for p in products]
print(len(product_list))
print(product_list[:5])

or_price_list = [op.text for op in or_prices]
print(len(or_price_list))
print(or_price_list[:5])

100
100
100
97
100
['에스트라', '에스트라', '에스트라', '바이오힐보', '구달']
100
['[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)', '[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획', '[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + 에이시카 세럼 3ML)', '[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml)', '[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획']
100
['59,400', '33,000', '53,400', '49,500', '50,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['24,000', '28,900', '42,000', '24,500', '27,360']


In [ ]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))


In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['스킨케어', '스킨케어', '스킨케어', '스킨케어', '스킨케어']
['크림', '에센스/세럼/앰플', '로션', '에센스/세럼/앰플', '에센스/세럼/앰플']
['크림', '에센스/세럼/앰플', '로션', '에센스/세럼/앰플', '에센스/세럼/앰플']
['(25,321건)', '(1,957건)', '(2,688건)', '(2,226건)', '(28,846건)']
['4.9', '4.8', '4.9', '4.7', '4.8']


In [ ]:
# 5. 데이터 저장
olive_skincare_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

NameError: name 'title_list' is not defined

In [ ]:
# DF 변환
olive_skincare_df = pd.DataFrame.from_dict(olive_skincare_dic, orient='index')
olive_skincare_df = olive_skincare_df.transpose()
olive_skincare_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_skincare_df.loc[olive_skincare_df['dis_price']=='']['product'].nunique())
olive_skincare_df.loc[olive_skincare_df['dis_price']=='']

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
31,아벤느,[장벽/보습]아벤느 트릭세라 뉴트리-플루이드 로션 200ml 더블 기획 *윤광 *광...,"44,000",,스킨케어,로션,로션,"(2,042건)",4.7
68,에스티로더,에스티로더 갈색병 30ml 기획 (갈색병 21ml 추가 증정),"127,000",,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,882건)",4.8
82,디오디너리,디오디너리 나이아신아마이드 10% + 징크 1% 30ml,"8,700",,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(10,438건)",4.6


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_skincare_df.reset_index(inplace=True)
olive_skincare_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,1,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,2,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,3,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,4,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_skincare_df.rename(columns={'index':'ranking'}, inplace=True)
olive_skincare_df['ranking'] = olive_skincare_df['ranking'].apply(lambda x : x+1)
olive_skincare_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,2,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,3,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,4,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,5,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
# 원본 데이터 저장
# olive_skincare_df.to_csv("oliveyoung_skincare_ranking.csv", index=False, encoding="utf-8-sig")

## 2. 마스크팩 랭킹

In [ ]:
# 1. 마스크팩 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010009&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%88%EC%8A%A4%ED%81%AC%ED%8C%A9")
time.sleep(3)  # 페이지 로드 대기

In [ ]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
91


In [ ]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['더마토리', '더마토리', '더마토리', '메디힐', '메디힐']
['[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획', '[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획', '[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트', '[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기', '[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1']
['26,000', '26,000', '26,000', '39,900', '2,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['17,600', '17,600', '17,600', '28,900', '1,000']


In [ ]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))


In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['마스크팩', '마스크팩', '마스크팩', '마스크팩', '마스크팩']
['패드', '패드', '패드', '패드', '시트팩']
['패드', '패드', '패드', '패드', '시트팩']
['(8건)', '(14건)', '(143건)', '(26,581건)', '(229,962건)']
['4.6', '4.9', '4.9', '4.7', '4.9']


In [ ]:
# 5. 데이터 저장
olive_mask_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [ ]:
# DF 변환
olive_mask_df = pd.DataFrame.from_dict(olive_mask_dic, orient='index')
olive_mask_df = olive_mask_df.transpose()
olive_mask_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_mask_df.loc[olive_mask_df['dis_price']=='']['product'].nunique())
olive_mask_df.loc[olive_mask_df['dis_price']=='']

91
9


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
33,스킨푸드,[700만개 판매/단독기획] 스킨푸드 캐롯 카로틴 카밍 워터 패드 기획 (60매+1...,"26,000",,마스크팩,패드,패드,"(61,247건)",4.8
41,비플레인,[3분/모공각질진정팩] 비플레인 녹두 모공 클레이 팩 120ml (+12ml*4ea...,"24,000",,마스크팩,페이셜팩,워시오프팩,"(11,248건)",4.7
60,브링그린,브링그린 프레시마스크 1매 5종,"1,800",,마스크팩,시트팩,시트팩,"(45,304건)",4.8
62,브링그린,[1+1] 브링그린 티트리시카포어코팩 5매입,"11,800",,마스크팩,코팩,코팩,"(10,227건)",4.8
68,메디필,메디필 랩핑 마스크팩 70mL 브러시 기획세트 2종 택1(레드락토 콜라겐/히알루론산...,"25,000",,마스크팩,페이셜팩,필오프팩,"(6,548건)",4.7
69,유세린,[데일리진정장벽] 유세린 울트라 센시티브 리페어 마스크팩 5매,"30,000",,마스크팩,시트팩,시트팩,"(1,802건)",4.8
74,메디힐,[26관왕/1위] 메디힐 더마 모델링 팩 풀패키지 5종 골라담기,"5,000",,마스크팩,페이셜팩,모델링팩,"(16,912건)",4.8
89,메디앤서,[모공쿨링/대용량]메디앤서 포어 콜라겐 나노샷 마스크 10매,"60,000",,마스크팩,시트팩,시트팩,(16건),4.4
90,에스트라,[NEW] 에스트라 에이시카365 쿨링팩패드 pH4.5 60매,"28,000",,마스크팩,패드,패드,(5건),5.0


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_mask_df.reset_index(inplace=True)
olive_mask_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,1,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,2,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,3,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,4,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_mask_df.rename(columns={'index':'ranking'}, inplace=True)
olive_mask_df['ranking'] = olive_mask_df['ranking'].apply(lambda x : x+1)
olive_mask_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,2,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,3,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,4,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,5,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [ ]:
# 원본 데이터 저장
# olive_mask_df.to_csv("oliveyoung_mask_ranking.csv", index=False, encoding="utf-8-sig")

## 3. 클렌징 랭킹

In [ ]:
# 1. 클렌징 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010010&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%ED%81%B4%EB%A0%8C%EC%A7%95")
time.sleep(3)  # 페이지 로드 대기

In [ ]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

# 4-1. 데이터 개수 확인
print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
88


In [ ]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['휩드', '마녀공장', '바이오더마', '비플레인', '바닐라코']
['[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽)', '[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획', '[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백)', '[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획', '[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처키트 7ml*4개입)']
['29,000', '37,000', '37,500', '21,900', '20,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])


100
['22,900', '25,900', '22,900', '18,600', '14,800']


In [ ]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['클렌징', '클렌징', '클렌징', '클렌징', '클렌징']
['클렌징폼/젤', '오일/밤', '워터/밀크', '클렌징폼/젤', '오일/밤']
['클렌징폼/젤', '클렌징오일', '클렌징워터', '클렌징폼/젤', '클렌징밤']
['(2,162건)', '(13,929건)', '(82,075건)', '(10,743건)', '(4,121건)']
['4.8', '4.9', '4.9', '4.8', '4.9']


In [ ]:
# 5. 데이터 저장
olive_clean_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [ ]:
# DF 변환
olive_clean_df = pd.DataFrame.from_dict(olive_clean_dic, orient='index')
olive_clean_df = olive_clean_df.transpose()
olive_clean_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_clean_df.loc[olive_clean_df['dis_price']=='']['product'].nunique())
olive_clean_df.loc[olive_clean_df['dis_price']=='']

88
12


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
45,폰즈,[나띠 PICK] 폰즈 클리어 스파 립앤아이 리무버 300ml,"24,500",,클렌징,립&아이리무버,립&아이리무버,"(81,629건)",4.9
50,메디필,[모공/약알칼리성] 메디필 레드 락토 콜라겐 클리어 폼 2.0 기획 (300ml+딥...,"24,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(16,578건)",4.7
51,메이크프렘,메이크프렘 세이프 미 릴리프 모이스처 클렌징밀크 200ml 기획(+100ml 리필),"28,000",,클렌징,워터/밀크,클렌징밀크/크림,"(7,137건)",4.8
52,비오텀,비오텀 클렌징 밀크 200ml,"27,800",,클렌징,워터/밀크,클렌징밀크/크림,"(3,207건)",4.8
53,아크네스,[촉촉클렌저/거품] 아크네스 포밍워시 200㎖,"16,500",,클렌징,클렌징폼/젤,클렌징폼/젤,"(3,510건)",4.7
69,메디필,[트러블/약산성] 메디필 그린 시카 콜라겐 클리어 폼 2.0 기획,"24,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(4,832건)",4.8
75,런드리유,[모공브러쉬 증정/단독기획] 런드리유 클린 페이스 가글 스트롱 젤 투 폼 클렌저 기...,"22,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(11,351건)",4.9
78,일소,[2024 어워즈] 일소 슈퍼 멜팅 세범 소프트너 150ml 한정기획 (+솜40매+...,"28,900",,클렌징,필링&스크럽,피지클리너,"(13,429건)",4.7
79,브링그린,[NEW] 브링그린 티트리시카딥클렌징폼 120mL,"10,900",,클렌징,클렌징폼/젤,클렌징폼/젤,(149건),4.8
87,이즈앤트리,[단독기획] 이즈앤트리 참마 비건 밀크 클렌저 220ml 리필 기획 (+리필100ml),"25,000",,클렌징,워터/밀크,클렌징밀크/크림,"(4,262건)",4.8


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_clean_df.reset_index(inplace=True)
olive_clean_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,1,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,2,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,3,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,4,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_clean_df.rename(columns={'index':'ranking'}, inplace=True)
olive_clean_df['ranking'] = olive_clean_df['ranking'].apply(lambda x : x+1)
olive_clean_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,2,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,3,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,4,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,5,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [ ]:
# 원본 데이터 저장
# olive_clean_df.to_csv("oliveyoung_cleansing_ranking.csv", index=False, encoding="utf-8-sig")

## 4. 선케어 랭킹

In [ ]:
# 선케어 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010011&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%84%A0%EC%BC%80%EC%96%B4")
time.sleep(3)  # 페이지 로드 대기

In [ ]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
76


In [ ]:
# 4-2. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['달바', '이지듀', '구달', '라운드랩', '라로슈포제']
['[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml)', '[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g', '[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획', '[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml)', '[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+시카토너50ml+시카밤3ml 증정)']
['51,000', '39,800', '20,000', '25,000', '34,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['35,700', '36,900', '17,500', '17,500', '25,000']


In [ ]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['선케어', '선케어', '선케어', '선케어', '선케어']
['선크림', '선쿠션', '선크림', '선크림', '선크림']
['선크림', '선쿠션', '선크림', '선크림', '선크림']
['(7,408건)', '(175건)', '(28,908건)', '(14,207건)', '(137건)']
['4.7', '4.6', '4.8', '4.8', '4.7']


In [ ]:
# 5. 데이터 저장
olive_suncare_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [ ]:
# DF 변환
olive_suncare_df = pd.DataFrame.from_dict(olive_suncare_dic, orient='index')
olive_suncare_df = olive_suncare_df.transpose()
olive_suncare_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_suncare_df.loc[olive_suncare_df['dis_price']=='']['product'].nunique())
olive_suncare_df.loc[olive_suncare_df['dis_price']=='']

76
24


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
9,넘버즈인,[더블기획] 넘버즈인 3번 도자기결 파데스킵 톤업베이지 35ml+35ml (SPF5...,"26,000",,선케어,선크림,선크림,"(10,772건)",4.6
14,넘버즈인,[1+1 기획] 넘버즈인 1번 청초 진정맑은 물막선크림 50ml+50ml 듀오기획 ...,"26,000",,선케어,선크림,선크림,"(5,105건)",4.8
24,구달,[고윤정PICK/1+1기획] 구달 맑은 어성초 진정 수분 톤업 선크림 50ml 1+1기획,"22,000",,선케어,선크림,선크림,"(4,016건)",4.6
28,셀퓨전씨,[UV에이징케어/잡티톤업썬] 셀퓨전씨 잡티 토닝 썬스크린 35ml 더블기획,"27,900",,선케어,선크림,선크림,(129건),4.9
33,달바,[수분자차/촉촉보습썬] 달바 에센스 선크림 듀오 기획 (50ml+50ml),"51,000",,선케어,선크림,선크림,"(7,178건)",4.8
36,셀퓨전씨,[UV에이징케어/밀착차단] 셀퓨전씨 레이저 UV 썬스크린 35ml 1+1 기획세트,"29,400",,선케어,선크림,선크림,"(2,452건)",4.8
44,브링그린,브링그린 티트리시카톤업선쿠션 15g_NEW,"20,000",,선케어,선쿠션,선쿠션,"(1,862건)",4.6
45,브링그린,브링그린 티트리시카쿨링선스틱 20g_NEW,"20,000",,선케어,선스틱,선스틱,(731건),4.6
47,스킨1004,[수분 선크림]스킨1004 마다가스카르 센텔라 히알루-시카 워터핏 선세럼 50ml,"19,000",,선케어,선크림,선크림,"(3,501건)",4.9
51,에스트라,에스트라 더마UV365 장벽수분 무기자차 선크림 40ml+20ml 기획,"31,000",,선케어,선크림,선크림,"(3,545건)",4.8


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_suncare_df.reset_index(inplace=True)
olive_suncare_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,1,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,2,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,3,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,4,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_suncare_df.rename(columns={'index':'ranking'}, inplace=True)
olive_suncare_df['ranking'] = olive_suncare_df['ranking'].apply(lambda x : x+1)
olive_suncare_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,2,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,3,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,4,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,5,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


## 5. 메이크업 랭킹

In [ ]:
# 메이크업업 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010002&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A9%94%EC%9D%B4%ED%81%AC%EC%97%85")
time.sleep(3)  # 페이지 로드 대기

In [ ]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
80


In [ ]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['어뮤즈', '3CE', '롬앤', '에스쁘아', '에스쁘아']
['[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (+리필+퍼프 증정)', '[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품)', '[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color', '[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) 8 colors + 미니 파우더', '[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors']
['38,000', '22,000', '13,000', '40,000', '22,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['28,500', '16,500', '9,900', '28,900', '16,500']


In [ ]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['메이크업', '메이크업', '메이크업', '메이크업', '메이크업']
['베이스메이크업', '립메이크업', '립메이크업', '베이스메이크업', '립메이크업']
['쿠션', '립스틱', '립틴트', '쿠션', '립틴트']
['(9,122건)', '(1,153건)', '(152,984건)', '(13,489건)', '(8,504건)']
['4.6', '4.7', '4.7', '4.7', '4.8']


In [ ]:
# 5. 데이터 저장
olive_makeup_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [ ]:
# DF 변환
olive_makeup_df = pd.DataFrame.from_dict(olive_makeup_dic, orient='index')
olive_makeup_df = olive_makeup_df.transpose()
olive_makeup_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_makeup_df.loc[olive_makeup_df['dis_price']=='']['product'].nunique())
olive_makeup_df.loc[olive_makeup_df['dis_price']=='']

80
20


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
12,오브제,[2024 어워즈] 오브제 커버로션 50g 어워즈 한정기획 (+미니어처 10g),"29,800",,메이크업,베이스메이크업,BB/CC,"(1,201건)",4.8
14,클리오,[NO.1 아이라이너] 클리오 샤프 쏘 심플 워터프루프 펜슬라이너,"11,000",,메이크업,아이메이크업,아이라이너,"(103,909건)",4.7
16,투크,[2024 어워즈/NEW 컬러 출시] 투크 워터프루프 슬림 아이라이너 16colors,"13,900",,메이크업,아이메이크업,아이라이너,"(30,474건)",4.7
29,하트퍼센트,[NEW/단독기획] 하트퍼센트 도트 온 무드 립펜슬 20colors (기획/단품),"14,000",,메이크업,립메이크업,립스틱,"(27,244건)",4.7
37,정샘물,[NEW] 정샘물 스킨 세팅 베이스 6종,"33,000",,메이크업,베이스메이크업,프라이머/베이스,"(1,890건)",4.6
39,VDL,"VDL 커버스테인 퍼펙팅 파운데이션 30ml (+휴대용파데, 퍼프기획/ 단품)","30,000",,메이크업,베이스메이크업,파운데이션,"(9,338건)",4.7
60,엔트로피 메이크업,[NEW 출시] 엔트로피 메이크업 브로우 컬러 애쉬브라운,"45,000",,메이크업,아이메이크업,아이브로우,"(1,288건)",4.7
63,지베르니,[NEW] 지베르니 플러피 듀얼 블러셔 3 COLORS (LAZY BUNNY) (온),"19,000",,메이크업,베이스메이크업,블러셔,(0건),0
64,페리페라,페리페라 잉크 무드 글로이 틴트,"11,000",,메이크업,립메이크업,립틴트,"(83,546건)",4.7
66,데이지크,[2024어워즈/3년연속수상] 데이지크 섀도우팔레트 24 colors (단품/기획),"34,000",,메이크업,아이메이크업,아이섀도우,"(29,268건)",4.7


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_makeup_df.reset_index(inplace=True)
olive_makeup_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,1,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,2,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,3,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,4,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_makeup_df.rename(columns={'index':'ranking'}, inplace=True)
olive_makeup_df['ranking'] = olive_makeup_df['ranking'].apply(lambda x : x+1)
olive_makeup_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,2,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,3,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,4,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,5,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [ ]:
# 원본 데이터 저장
# olive_makeup_df.to_csv("oliveyoung_makeup_ranking.csv", index=False, encoding="utf-8-sig")

In [162]:
driver.quit()

NameError: name 'driver' is not defined

## 데이터셋 공통 전처리_통합 버전

In [ ]:
oliveyoung = pd.concat([olive_skincare_df, olive_clean_df, olive_mask_df, olive_suncare_df, olive_makeup_df], axis=0, ignore_index=True)
print(oliveyoung.info())
oliveyoung.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    object 
 4   dis_price     432 non-null    object 
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    object 
 9   review_score  500 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB
None


,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,2,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,3,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,4,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,5,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
a = '테스트용'

a_253 = a.split(']')
max_a253 = len(a_253)-1

print(a_253[0], len(a_253))


b = oliveyoung['product'][0]

b_253 = b.split(']')
max_b253 = len(b_253)-1

print(b_253[0][0], '/', b_253, b_253[max_b253], '/', len(b_253))


c = ' 테스트용[ dddd]'

c_253 = c.split(']')
max_c253 = len(c_253)-1
c_result = c_253[0]
c_second = c_result.split('[')
c_result2 = c_second[0]

print(f"'{c_253[0].strip()}'", '/', c_253, c_253[max_c253], '/', len(c_253), c_result2)


d = oliveyoung['product'][60]

d_253 = d.split(']')
max_d253 = len(d_253)-1

print(d_253, d_253[max_d253], '/', len(d_253))

테스트용 1
[ / ['[2월올영픽/1등크림', ' 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)']  에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML) / 2
'테스트용[ dddd' / [' 테스트용[ dddd', '']  / 2  테스트용
['[아이소이x망그러진곰', '[대용량', ' 아이소이 모이스춰 닥터 장수진 수분크림100ml 한정기획(+키링&장바구니증정)']  아이소이 모이스춰 닥터 장수진 수분크림100ml 한정기획(+키링&장바구니증정) / 3


In [ ]:
# 1. 상품명 정리 - 신버전 
for i in range(len(oliveyoung)):
  
  name_list = oliveyoung['product'][i].split(']')
  max_i = len(name_list)-1
  
  if (len(name_list) == 2):
    if (name_list[0][0] == '['):
      if name_list[1] == '': # 3)의 경우
        name_result = name_list[0]
        name_result = name_result.strip('[')
        name_result = name_result.strip()
        oliveyoung['product'][i] = name_result
      else: # 1)의 경우 
        name_result = name_list[1]
        name_result = name_result.strip()
        oliveyoung['product'][i] = name_result
    
    else: # 2)의 경우
      name_result = name_list[0]
      second_list = name_result.split('[')
      name_result2 = second_list[0]
      oliveyoung['product'][i] = name_result2
  
  elif (len(name_list)==1): # 5)의 경우 
    name_result = name_list[0]
    oliveyoung['product'][i] = name_result
  
  else: # 길이가 3일 경우
    if name_list[2] == '': # 4)의 경우 
      name_result = name_list[1]
      second_list = name_result.split('[')
      name_result2 = second_list[0]
      oliveyoung['product'][i] = name_result2
    
    else: # 6의 경우 
      name_result = name_list[2]
      name_result = name_result.strip()
      oliveyoung['product'][i] = name_result

print(oliveyoung['product'].unique())     
  
  # 경우의 수
  # 1) [] 가 맨 앞에 있을 경우('[', '상품명') -> 리스트의 마지막 글자 사용 name_result = name_list[i][max_i] -> name_result.strip() => 상품명
  # 3) [] 안이 상품명일 경우('[상품명','') -> name_result = name_list[i][0] -> name_result.strip('[') & strip() => 상품명
  
  # 2) [] 가 맨 뒤에 있을 경우('상품명[~', '') -> 리스트의 첫 글자 사용 name_result = name_list[i][0] => 상품명[~
  
  # 4) [] 가 앞뒤로 있을 경우('[~','상품명[~','') -> name_result = name_list[i][max_1-1] => 상품명[~
  
  # 5) [] 가 아예 없는 경우 
  
  # 6) []가 두번 연속 있을 경우 

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

['에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)'
 '에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획'
 '에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + 에이시카 세럼 3ML)'
 '바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml)'
 '구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획'
 '바이오더마 시카비오 크림+ 100ml 기획 (+40ml 증정)'
 '토리든 다이브인 저분자 히알루론산 세럼 50ml 리필기획(+리필팩 50ml)' '에스트라 아토베리어365 크림미스트 120ml'
 '닥터자르트 시카페어 인텐시브 수딩 리페어 세럼 30ml 1+1 기획'
 '라로슈포제 시카플라스트 밤 B5+ 200ml 기획 (+시카PRO마스크 1매)'
 '차앤박 더마앤서 액티브 부스트 PDRN앰플 30ml 기획 (+15ml 증정)'
 '웰라쥬 리얼 히알루로닉 블루 100 앰플 75ml 1+1 기획' '바이오더마 시카비오 포마드 100ml(리페어 리치 밤)'
 '바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml+30ml 더블기획' '셀라딕스 세범 리밸런싱 RX 131 앰플'
 '차앤박 프로폴리스 에너지 액티브 앰플 30ml 트리플 기획'
 '에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획(60ml+60ml)'
 '닥터지 레드 블레미쉬 클리어 수딩 크림 기획세트 (100ml/150ml구성)'
 '바이오더마 하이드라비오 에센스 로션 200ml(화장솜 증정)'
 '라네즈 크림스킨 170ml 리필기획 (+170ml 리필+미스트펌프 증정)'
 '바이오힐보 프로바이오덤 3D 리프팅 크림 50ml 리필기획 (+윤곽마스크 2매)(올리지오 콜라보)'
 '메디큐브 연어 PDRN 핑크 앰플 더블기획 (30ml+30ml)'
 '라로슈포제 시카플라스트 크림 B5 100ml 기획 (+시카토너 50ml+시카크림 3ml 증정)'
 '파

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28600\511079030.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
# 결측치 더블체크
print(oliveyoung[oliveyoung['product'] == ''])
print(oliveyoung[oliveyoung['product'] == ' '])

Empty DataFrame
Columns: [ranking, brand, product, or_price, dis_price, main_class, mid_class, sub_class, review_num, review_score]
Index: []
Empty DataFrame
Columns: [ranking, brand, product, or_price, dis_price, main_class, mid_class, sub_class, review_num, review_score]
Index: []


In [ ]:
# 2. 리뷰 수 서식 지우기

for i in range(len(oliveyoung)):
  name = oliveyoung['review_num'][i]

  name_result = name.strip('(''건'')')
  name_result = name_result.replace(',', '')
  oliveyoung.loc[i, 'review_num'] = name_result

print(oliveyoung.loc[:5, 'review_num'])

0    25321
1     1957
2     2688
3     2226
4    28846
5     3561
Name: review_num, dtype: object


In [ ]:
# 결측값이 float 형태로 읽히는 것을 방지하기 위해 결측값 공백으로 대체
oliveyoung['dis_price'].fillna('', inplace=True)

oliveyoung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    object 
 4   dis_price     500 non-null    object 
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    object 
 9   review_score  500 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB


In [ ]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 1) price 컬럼의 쉼표 없애기

for i in range(len(oliveyoung)):
  or_p = oliveyoung['or_price'][i]
  dis_p = oliveyoung['dis_price'][i]
  
  or_result = or_p.replace(',', '')
  dis_result = dis_p.replace(',', '')
  
  oliveyoung.loc[i, 'or_price'] = or_result
  oliveyoung.loc[i, 'dis_price'] = dis_result

print(oliveyoung.loc[:5, ['or_price', 'dis_price', 'review_num']])

  or_price dis_price review_num
0    59400     44500      25321
1    33000     24700       1957
2    53400     39900       2688
3    49500     27300       2226
4    50000     32900      28846
5    43000     33900       3561


In [ ]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 2) 컬럼 타입 변경

oliveyoung[['or_price', 'dis_price', 'review_num', 'review_score']] = oliveyoung[['or_price', 'dis_price', 'review_num', 'review_score']].apply(pd.to_numeric)
print(oliveyoung[['or_price', 'dis_price', 'review_num', 'review_score']].dtypes)
print(oliveyoung.loc[:5, ['or_price', 'dis_price', 'review_num', 'review_score']])

or_price          int64
dis_price       float64
review_num        int64
review_score    float64
dtype: object
   or_price  dis_price  review_num  review_score
0     59400    44500.0       25321           4.9
1     33000    24700.0        1957           4.8
2     53400    39900.0        2688           4.9
3     49500    27300.0        2226           4.7
4     50000    32900.0       28846           4.8
5     43000    33900.0        3561           4.8


In [ ]:
# 4. 플랫폼 컬럼 삽입 : df['플랫폼'] = '올리브영'
oliveyoung['플랫폼'] = '올리브영'
print(oliveyoung.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    int64  
 4   dis_price     432 non-null    float64
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    int64  
 9   review_score  500 non-null    float64
 10  플랫폼           500 non-null    object 
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None


In [ ]:
# 5. 컬럼명 및 순서 변경 : 플랫폼-브랜드-제품명-대분류_순위-대분류-중분류-소분류-평점-후기 수-가격
# 1) 컬럼명 변경
oliveyoung.columns = ['중분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점', '플랫폼']
print(oliveyoung.columns)

Index(['중분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')


In [ ]:
oliveyoung = oliveyoung.reindex(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '정가', '할인가'], axis=1)

print(oliveyoung.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None


In [ ]:
# 데이터셋 저장
# olive_df.to_csv('통합_올리브영_데이터.csv', encoding="utf-8-sig", index=False)

# 데이터 전처리

## 무신사뷰티

In [142]:
# 데이터셋 복제
musinsa_df = musinsa.copy(deep=True)

In [143]:
# 데이터셋 기본 정보 조회
print(musinsa_df.info())
print(musinsa_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      400 non-null    int64 
 1   카테고리    400 non-null    object
 2   중분류     400 non-null    object
 3   소분류     400 non-null    object
 4   브랜드     400 non-null    object
 5   제품명     400 non-null    object
 6   평점      400 non-null    object
 7   후기 수    400 non-null    object
 8   가격      400 non-null    object
 9   URL     400 non-null    object
dtypes: int64(1), object(9)
memory usage: 31.4+ KB
None
   순위 카테고리     중분류   소분류    브랜드  \
0   1  클렌징  페이셜클렌징  클렌징폼  이니스프리   
1   2  클렌징  페이셜클렌징  클렌징폼   비플레인   
2   3  클렌징  페이셜클렌징  클렌징폼  에스네이처   

                                                 제품명   평점   후기 수      가격  \
0                           화산송이 모공 바하 클렌징 폼 150g 2개  4.9  7,915  18,200   
1              [대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정)  4.9  2,870  31,200   
2  아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징

### 기본 전처리

In [144]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# + 평점 컬럼의 '별점 없음' 을 0으로 변경

for i in range(len(musinsa_df)):
  score = musinsa_df['평점'][i]
  review = musinsa_df['후기 수'][i]
  price = musinsa_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  musinsa_df.loc[i, '평점'] = score_result
  musinsa_df.loc[i, '후기 수'] = review_result
  musinsa_df.loc[i, '가격'] = price_result

print(musinsa_df.loc[:10, ['평점', '후기 수', '가격']])

     평점  후기 수     가격
0   4.9  7915  18200
1   4.9  2870  31200
2   4.9   630  31900
3   4.9  3707  23000
4   4.9  1215  23000
5   4.9   990  23000
6   4.9  1004  12900
7   4.8   217  29000
8   4.9  5188  19500
9   4.9  1376  17400
10  4.8  1930  21000


In [145]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경

musinsa_df[['평점', '후기 수', '가격']] = musinsa_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(musinsa_df[['평점', '후기 수', '가격']].dtypes)
print(musinsa_df.loc[:5, ['평점', '후기 수', '가격']])

평점      float64
후기 수      int64
가격        int64
dtype: object
    평점  후기 수     가격
0  4.9  7915  18200
1  4.9  2870  31200
2  4.9   630  31900
3  4.9  3707  23000
4  4.9  1215  23000
5  4.9   990  23000


In [146]:
# 2. 컬럼 이름 변경
musinsa_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)
musinsa_df.columns

Index(['중분류_순위', '대분류', '중분류', '소분류', '브랜드', '제품명', '평점', '후기 수', '가격', 'URL'], dtype='object')

In [147]:
# 3. 플랫폼 컬럼 추가 및 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '정가', '할인가'
musinsa_df['플랫폼'] = '무신사뷰티'

before_a = ['플랫폼']
a = ['브랜드', '제품명']
after_a = musinsa_df.drop(columns = {'플랫폼', '브랜드', '제품명'}).columns[:].to_list()

reorder = before_a + a + after_a
musinsa_df = musinsa_df[reorder]

musinsa_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '가격',
       'URL'],
      dtype='object')

### 대분류 및 중분류 통합

In [148]:
# 1. 중분류1 & 중분류2 컬럼 생성
musinsa_df['중분류1'] = ''
print(musinsa_df['중분류1'].head())

musinsa_df['중분류2'] = ''
print(musinsa_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [149]:
# 기존 분류별 unique 값 확인
print(musinsa_df['대분류'].unique())
print(musinsa_df['중분류'].unique())
print(musinsa_df['소분류'].unique())

['클렌징' '스킨케어' '썬케어' '메이크업']
['페이셜클렌징' '메이크업클렌징' '필링/스크럽' '립&아이 리무버' '에센스/세럼/앰플' '기획세트' '마스크팩' '크림'
 '패드' '로션' '미스트' '스킨/토너' '올인원' '스팟케어' '선크림' '선세럼/젤' '선쿠션/파우더' '선스틱/선스프레이'
 '태닝/애프터선' '선케어세트' '립메이크업' '베이스메이크업' '아이메이크업']
['클렌징폼' '클렌징로션/크림' '클렌징오일' '필링' '없음' '클렌징패드' '페이스 스크럽' '클렌징워터' '바디 스크럽'
 '클렌징젤' '카테고리 없음' '페이셜팩' '코팩/패치' '선세럼' '선스틱' '애프터선' '태닝' '선젤' '립틴트' '쿠션'
 '파운데이션' '하이라이터' '립밤' '립라이너' '셰이딩/컨투어링' '파우더/콤팩트' '아이라이너' '블러셔' '컨실러'
 '메이크업 베이스' '립글로스' '아이섀도' '립스틱' '마스카라' 'BB크림/CC크림' '아이브로' '메이크업 픽서']


In [150]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = musinsa_df['중분류'] == '기획세트'
print(musinsa_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = musinsa_df['중분류'] == '립&아이 리무버'
print(musinsa_df.loc[cond2, '대분류'].unique()) # 클렌징 -> 기타에서 제외

cond3 = musinsa_df['중분류'] == '미스트'
print(musinsa_df.loc[cond3, '대분류'].unique()) # 스킨케어

cond4 = musinsa_df['중분류'] == '올인원'
print(musinsa_df.loc[cond4, '대분류'].unique()) # 스킨케어 

cond5 = musinsa_df['중분류'] == '스팟케어'
print(musinsa_df.loc[cond5, '대분류'].unique()) # 스킨케어 

['스킨케어']
['클렌징']
['스킨케어']
['스킨케어']
['스킨케어']


In [151]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['패드', '스킨/토너']
skin = '|'.join(skin_list)
cond_skin = musinsa_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = musinsa_df['소분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '픽서']
fixer = '|'.join(fixer_list)
cond_fixer = musinsa_df['소분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = musinsa_df['소분류'].str.contains(shading)

primer_list = ['프라이머', '베이스']
primer = '|'.join(primer_list)
cond_primer = musinsa_df['소분류'].str.contains(primer)

for i in range(len(musinsa_df)):
  if cond_skin[i] == True:
    musinsa_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    musinsa_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    musinsa_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    musinsa_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    musinsa_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if musinsa_df['대분류'][i] == '클렌징':
      musinsa_df.loc[i, '중분류2'] = '클렌징'

    elif musinsa_df['중분류'][i] == '에센스/세럼/앰플':
      musinsa_df.loc[i, '중분류2'] = '에센스/세럼/앰플'

    elif musinsa_df['중분류'][i] == '마스크팩':
      musinsa_df.loc[i, '중분류2'] = '마스크팩'
      
    elif musinsa_df['중분류'][i] == '크림':
      musinsa_df.loc[i, '중분류2'] = '크림'

    elif musinsa_df['중분류'][i] == '로션':
      musinsa_df.loc[i, '중분류2'] = '로션'

    elif musinsa_df['대분류'][i] == '썬케어':
      musinsa_df.loc[i, '중분류2'] = '선케어'

    elif musinsa_df['소분류'][i] == '컨실러':
      musinsa_df.loc[i, '중분류2'] = '컨실러'

    elif musinsa_df['중분류'][i] == '립메이크업':
      musinsa_df.loc[i, '중분류2'] = '립메이크업'
      
    elif musinsa_df['소분류'][i] == '아이라이너':
      musinsa_df.loc[i, '중분류2'] = '아이라이너'

    elif musinsa_df['소분류'][i] == '아이섀도':
      musinsa_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif musinsa_df['소분류'][i] == '아이브로':
      musinsa_df.loc[i, '중분류2'] = '아이브로우'

    elif musinsa_df['소분류'][i] == '마스카라':
      musinsa_df.loc[i, '중분류2'] = '마스카라'
      
    else: # 기획세트
      musinsa_df.loc[i, '중분류2'] = '기타'

In [152]:
print(musinsa_df['중분류2'].unique())
print(musinsa_df['중분류2'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['클렌징' '에센스/세럼/앰플' '기타' '마스크팩' '크림' '토너/토너패드/스킨' '로션' '선케어' '립메이크업'
 '파운데이션/쿠션/BB/CC' '블러셔/쉐이딩/하이라이터' '파우더/팩트/픽서' '아이라이너' '컨실러' '프라이머/베이스'
 '아이섀도우' '마스카라' '아이브로우']
18


,대분류,중분류,소분류,중분류2
0,클렌징,페이셜클렌징,클렌징폼,클렌징
1,클렌징,페이셜클렌징,클렌징폼,클렌징
2,클렌징,페이셜클렌징,클렌징폼,클렌징
3,클렌징,페이셜클렌징,클렌징폼,클렌징
4,클렌징,메이크업클렌징,클렌징로션/크림,클렌징
...,...,...,...,...
395,메이크업,베이스메이크업,BB크림/CC크림,파운데이션/쿠션/BB/CC
396,메이크업,립메이크업,립틴트,립메이크업
397,메이크업,베이스메이크업,블러셔,블러셔/쉐이딩/하이라이터
398,메이크업,립메이크업,립틴트,립메이크업


In [153]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = musinsa_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = musinsa_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹 영양제']
eye = '|'.join(eye_list)
cond_eye = musinsa_df['중분류2'].str.contains(eye)

for i in range(len(musinsa_df)):
  if cond_skincare[i] == True:
    musinsa_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    musinsa_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    musinsa_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if musinsa_df['중분류2'][i] == '클렌징':
      musinsa_df.loc[i, '중분류1'] = '클렌징'

    elif musinsa_df['중분류2'][i] == '마스크팩':
      musinsa_df.loc[i, '중분류1'] = '마스크팩'
      
    elif musinsa_df['중분류2'][i] == '기타':
      musinsa_df.loc[i, '중분류1'] = '기타'
      
    elif musinsa_df['중분류2'][i] == '선케어':
      musinsa_df.loc[i, '중분류1'] = '선케어'
    
    elif musinsa_df['중분류2'][i] == '립메이크업':
      musinsa_df.loc[i, '중분류1'] = '립메이크업'

In [154]:
print(musinsa_df['중분류1'].unique())
print(musinsa_df['중분류1'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['클렌징' '스킨케어' '기타' '마스크팩' '선케어' '립메이크업' '베이스메이크업' '아이메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
1,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
2,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
3,클렌징,페이셜클렌징,클렌징폼,클렌징,클렌징
4,클렌징,메이크업클렌징,클렌징로션/크림,클렌징,클렌징
...,...,...,...,...,...
395,메이크업,베이스메이크업,BB크림/CC크림,베이스메이크업,파운데이션/쿠션/BB/CC
396,메이크업,립메이크업,립틴트,립메이크업,립메이크업
397,메이크업,베이스메이크업,블러셔,베이스메이크업,블러셔/쉐이딩/하이라이터
398,메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [155]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = musinsa_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = musinsa_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = musinsa_df['중분류1'].str.contains(colorclass)

for i in range(len(musinsa_df)):
  if cond_skinclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    musinsa_df.loc[i, '대분류'] = '색조메이크업'

In [156]:
# 대분류 고유값 확인 
print(musinsa_df['대분류'].unique())
print(musinsa_df['대분류'].nunique())
musinsa_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
1,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
2,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
3,스킨케어,페이셜클렌징,클렌징폼,클렌징,클렌징
4,스킨케어,메이크업클렌징,클렌징로션/크림,클렌징,클렌징
...,...,...,...,...,...
395,베이스메이크업,베이스메이크업,BB크림/CC크림,베이스메이크업,파운데이션/쿠션/BB/CC
396,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업
397,베이스메이크업,베이스메이크업,블러셔,베이스메이크업,블러셔/쉐이딩/하이라이터
398,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업


### 립메이크업 중분류 세분화

In [157]:
# musinsa_df
musinsa_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,URL,중분류1,중분류2
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,페이셜클렌징,클렌징폼,4.9,7915,18200,https://www.musinsa.com/products/3670796,클렌징,클렌징
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,페이셜클렌징,클렌징폼,4.9,2870,31200,https://www.musinsa.com/products/3138771,클렌징,클렌징
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,페이셜클렌징,클렌징폼,4.9,630,31900,https://www.musinsa.com/products/4566395,클렌징,클렌징
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,페이셜클렌징,클렌징폼,4.9,3707,23000,https://www.musinsa.com/products/3312180,클렌징,클렌징
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,메이크업클렌징,클렌징로션/크림,4.9,1215,23000,https://www.musinsa.com/products/3994035,클렌징,클렌징


In [158]:
# 중분류2 복제
musinsa_df['중분류3'] = musinsa_df['중분류2']
musinsa_df['중분류3'].head()

0    클렌징
1    클렌징
2    클렌징
3    클렌징
4    클렌징
Name: 중분류3, dtype: object

In [159]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = musinsa_df['중분류2'] == '립메이크업'
musinsa_df.loc[cond, '소분류'].unique()

array(['립틴트', '립밤', '립라이너', '립글로스', '립스틱'], dtype=object)

In [160]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

for i in range(len(musinsa_df)):
  if musinsa_df['소분류'][i] == '립글로스':
    musinsa_df.loc[i, '중분류3'] = '립글로스'

  elif musinsa_df['소분류'][i] == '립밤':
    musinsa_df.loc[i, '중분류3'] = '립밤'
    
  elif musinsa_df['소분류'][i] == '립스틱':
    musinsa_df.loc[i, '중분류3'] = '립스틱'
    
  elif musinsa_df['소분류'][i] == '립라이너':
    musinsa_df.loc[i, '중분류3'] = '립라이너'

  elif musinsa_df['소분류'][i] == '립틴트':
    musinsa_df.loc[i, '중분류3'] = '립틴트'

In [161]:
# 중분류3 입력값 확인
cond = musinsa_df['중분류2'] == '립메이크업'
print(musinsa_df.loc[cond, '중분류3'].unique())
print(musinsa_df.loc[cond, '중분류3'].nunique())
musinsa_df.loc[cond, ['소분류', '중분류3']].head()

['립틴트' '립밤' '립라이너' '립글로스' '립스틱']
5


,소분류,중분류3
300,립틴트,립틴트
302,립틴트,립틴트
308,립밤,립밤
310,립라이너,립라이너
312,립밤,립밤


### 데이터셋 정리

In [162]:
# 기존 카테고리 지우기
musinsa_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
musinsa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   평점      400 non-null    float64
 6   후기 수    400 non-null    int64  
 7   가격      400 non-null    int64  
 8   URL     400 non-null    object 
 9   중분류1    400 non-null    object 
 10  중분류2    400 non-null    object 
 11  중분류3    400 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.6+ KB


In [163]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = musinsa_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = musinsa_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
musinsa_df = musinsa_df[reorder]

musinsa_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', 'URL'],
      dtype='object')

In [164]:
# 최종 데이터셋 확인
print(musinsa_df.info())
musinsa_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   중분류1    400 non-null    object 
 6   중분류2    400 non-null    object 
 7   중분류3    400 non-null    object 
 8   평점      400 non-null    float64
 9   후기 수    400 non-null    int64  
 10  가격      400 non-null    int64  
 11  URL     400 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.6+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,URL
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,클렌징,클렌징,클렌징,4.9,7915,18200,https://www.musinsa.com/products/3670796
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,클렌징,클렌징,클렌징,4.9,2870,31200,https://www.musinsa.com/products/3138771
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,클렌징,클렌징,클렌징,4.9,630,31900,https://www.musinsa.com/products/4566395
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,클렌징,클렌징,클렌징,4.9,3707,23000,https://www.musinsa.com/products/3312180
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,클렌징,클렌징,클렌징,4.9,1215,23000,https://www.musinsa.com/products/3994035


In [ ]:
# 최종 데이터셋 저장
# musinsa_df.to_csv('musinsa_preprocessing.csv', index=False, encoding="utf-8-sig")

## 화해

In [167]:
# 데이터셋 복제
hwahae_df = hwahae.copy(deep=True)

In [168]:
# 데이터셋 기본 정보 조회
print(hwahae_df.info())
print(hwahae_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   중분류     4781 non-null   object 
 6   소분류     0 non-null      float64
 7   평점      4781 non-null   float64
 8   후기 수    4781 non-null   int64  
 9   가격      4686 non-null   float64
 10  URL     0 non-null      float64
dtypes: float64(4), int64(2), object(5)
memory usage: 411.0+ KB
None
  플랫폼    브랜드                                        제품명  중분류_순위   대분류   중분류  \
0  화해  에스네이처         아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜)       1  스킨케어  스킨토너   
1  화해    토리든  다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml)       2  스킨케어  스킨토너   
2  화해   라운드랩             1025 독도 토너 200ml + 독도 토너 500ml       3  스킨케어  스킨토너   

   소분류    평점   후기 수       

### 기본 전처리

In [169]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# 원본 데이터셋에 전처리 되어있음

'''
for i in range(len(hwahae_df)):
  score = hwahae_df['평점'][i]
  review = hwahae_df['후기 수'][i]
  price = hwahae_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  hwahae_df.loc[i, '평점'] = score_result
  hwahae_df.loc[i, '후기 수'] = review_result
  hwahae_df.loc[i, '가격'] = price_result

print(hwahae_df.loc[:10, ['평점', '후기 수', '가격']])
'''

"\nfor i in range(len(hwahae_df)):\n  score = hwahae_df['평점'][i]\n  review = hwahae_df['후기 수'][i]\n  price = hwahae_df['가격'][i]\n  \n  score_result = score.replace('별점 없음', '0')\n  review_result = review.replace(',', '')\n  price_result = price.replace(',', '')\n  \n  hwahae_df.loc[i, '평점'] = score_result\n  hwahae_df.loc[i, '후기 수'] = review_result\n  hwahae_df.loc[i, '가격'] = price_result\n\nprint(hwahae_df.loc[:10, ['평점', '후기 수', '가격']])\n"

In [170]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df[['평점', '후기 수', '가격']] = hwahae_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(hwahae_df[['평점', '후기 수', '가격']].dtypes)
print(hwahae_df.loc[:5, ['평점', '후기 수', '가격']])
'''

"\nhwahae_df[['평점', '후기 수', '가격']] = hwahae_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)\n\nprint(hwahae_df[['평점', '후기 수', '가격']].dtypes)\nprint(hwahae_df.loc[:5, ['평점', '후기 수', '가격']])\n"

In [171]:
# 2. 컬럼 이름 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)
hwahae_df.columns
'''

"\nhwahae_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)\nhwahae_df.columns\n"

In [172]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df['플랫폼'] = '화해'

a = hwahae_df.columns[-1:].to_list()

ex_a = hwahae_df.drop(columns = hwahae_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
hwahae_df = hwahae_df[reorder]

hwahae_df.columns
'''

"\nhwahae_df['플랫폼'] = '화해'\n\na = hwahae_df.columns[-1:].to_list()\n\nex_a = hwahae_df.drop(columns = hwahae_df.columns[[-1]]).columns[:].to_list()\n\nreorder = a + ex_a\nhwahae_df = hwahae_df[reorder]\n\nhwahae_df.columns\n"

### 카테고리 전처리

In [173]:
# 1. 중분류1 & 중분류2 컬럼 생성
hwahae_df['중분류1'] = ''
print(hwahae_df['중분류1'].head())

hwahae_df['중분류2'] = ''
print(hwahae_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [174]:
# 기존 분류별 unique 값 확인
print(hwahae_df['대분류'].unique())
print(hwahae_df['중분류'].unique())
print(hwahae_df['소분류'].unique())

['스킨케어' '클렌징' '마스크_팩' '선케어' '베이스메이크업' '아이메이크업' '립메이크업']
['스킨토너' '로션_에멀젼' '에센스_앰플_세럼' '페이스오일' '크림' '아이케어' '미스트' '젤' '스킨_토너패드'
 '밤_멀티밤' '클렌징폼' '클렌징워터' '클렌징젤' '클렌징오일' '클렌징로션_크림' '클렌징비누' '클렌징티슈_패드'
 '립_아이리무버' '스크럽_필링' '스크럽_필링패드' '클렌징파우더' '클렌징밤' '시트마스크' '부분마스크_팩' '워시오프팩'
 '필오프팩' '슬리핑팩' '패치' '코팩' '부분마스크패드' '선크림_로션' '선스프레이' '선케어기타' '선스틱' '선쿠션_팩트'
 '메이크업베이스' '프라이머' 'BB_CC크림' '파운데이션' '쿠션' '파우더_팩트' '컨실러' '블러셔' '하이라이터'
 '메이크업픽서' '셰이딩' '톤업크림' '아이섀도' '아이라이너' '아이브로우' '마스카라_픽서' '속눈썹영양제' '립스틱'
 '립틴트' '립글로스' '립케어_립밤' '컬러립케어_립밤']
[nan]


In [175]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = hwahae_df['중분류'] == '아이케어'
print(hwahae_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = hwahae_df['중분류'] == '미스트'
print(hwahae_df.loc[cond2, '대분류'].unique()) # 스킨케어

cond3 = hwahae_df['중분류'] == '젤'
print(hwahae_df.loc[cond3, '대분류'].unique()) # 스킨케어

['스킨케어']
['스킨케어']
['스킨케어']


In [176]:
# 통합 필요한 세부 카테고리 확인
cond1 = hwahae_df['대분류'] == '스킨케어'
print(hwahae_df.loc[cond1, '중분류'].unique()) 

cond2 = hwahae_df['대분류'] == '베이스메이크업'
print(hwahae_df.loc[cond2, '중분류'].unique()) 

cond3 = hwahae_df['대분류'] == '아이메이크업'
print(hwahae_df.loc[cond3, '중분류'].unique()) 

['스킨토너' '로션_에멀젼' '에센스_앰플_세럼' '페이스오일' '크림' '아이케어' '미스트' '젤' '스킨_토너패드'
 '밤_멀티밤']
['메이크업베이스' '프라이머' 'BB_CC크림' '파운데이션' '쿠션' '파우더_팩트' '컨실러' '블러셔' '하이라이터'
 '메이크업픽서' '셰이딩' '톤업크림']
['아이섀도' '아이라이너' '아이브로우' '마스카라_픽서' '속눈썹영양제']


In [177]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['패드', '스킨', '토너']
skin = '|'.join(skin_list)
cond_skin = hwahae_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = hwahae_df['중분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '메이크업픽서']
fixer = '|'.join(fixer_list)
cond_fixer = hwahae_df['중분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = hwahae_df['중분류'].str.contains(shading)

primer_list = ['프라이머', '베이스', '톤업크림'] # 프라이머/베이스 컬럼에 (소분류)톤업크림 추가
primer = '|'.join(primer_list)
cond_primer = hwahae_df['중분류'].str.contains(primer)

for i in range(len(hwahae_df)):
  if cond_skin[i] == True:
    hwahae_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    hwahae_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    hwahae_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    hwahae_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    hwahae_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if hwahae_df['중분류'][i] == '로션_에멀젼':
      hwahae_df.loc[i, '중분류2'] = '로션'

    elif hwahae_df['중분류'][i] == '에센스_앰플_세럼':
      hwahae_df.loc[i, '중분류2'] = '에센스/세럼/앰플'
    
    elif hwahae_df['중분류'][i] == '크림':
      hwahae_df.loc[i, '중분류2'] = '크림'

    elif hwahae_df['대분류'][i] == '마스크_팩':
      hwahae_df.loc[i, '중분류2'] = '마스크팩'

    elif hwahae_df['대분류'][i] == '클렌징':
      hwahae_df.loc[i, '중분류2'] = '클렌징'

    elif hwahae_df['대분류'][i] == '선케어':
      hwahae_df.loc[i, '중분류2'] = '선케어'

    elif hwahae_df['대분류'][i] == '립메이크업':
      hwahae_df.loc[i, '중분류2'] = '립메이크업'
    
    elif hwahae_df['중분류'][i] == '컨실러':
      hwahae_df.loc[i, '중분류2'] = '컨실러'
      
    elif hwahae_df['중분류'][i] == '아이라이너':
      hwahae_df.loc[i, '중분류2'] = '아이라이너'

    elif hwahae_df['중분류'][i] == '아이섀도':
      hwahae_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif hwahae_df['중분류'][i] == '아이브로우':
      hwahae_df.loc[i, '중분류2'] = '아이브로우'

    elif hwahae_df['중분류'][i] == '마스카라_픽서':
      hwahae_df.loc[i, '중분류2'] = '마스카라'
    
    elif hwahae_df['중분류'][i] == '속눈썹영양제':
      hwahae_df.loc[i, '중분류2'] = '속눈썹영양제'
      
    else: # 기획세트
      hwahae_df.loc[i, '중분류2'] = '기타'

In [178]:
print(hwahae_df['중분류2'].unique())
print(hwahae_df['중분류2'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['토너/토너패드/스킨' '로션' '에센스/세럼/앰플' '기타' '크림' '클렌징' '파우더/팩트/픽서' '마스크팩' '선케어'
 '파운데이션/쿠션/BB/CC' '프라이머/베이스' '컨실러' '블러셔/쉐이딩/하이라이터' '아이섀도우' '아이라이너' '아이브로우'
 '마스카라' '속눈썹영양제' '립메이크업']
19


,대분류,중분류,소분류,중분류2
0,스킨케어,스킨토너,NaN,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,토너/토너패드/스킨
...,...,...,...,...
4776,립메이크업,컬러립케어_립밤,NaN,립메이크업
4777,립메이크업,컬러립케어_립밤,NaN,립메이크업
4778,립메이크업,컬러립케어_립밤,NaN,립메이크업
4779,립메이크업,컬러립케어_립밤,NaN,립메이크업


In [179]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = hwahae_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = hwahae_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = hwahae_df['중분류2'].str.contains(eye)

for i in range(len(hwahae_df)):
  if cond_skincare[i] == True:
    hwahae_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    hwahae_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    hwahae_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if hwahae_df['중분류2'][i] == '클렌징':
      hwahae_df.loc[i, '중분류1'] = '클렌징'

    elif hwahae_df['중분류2'][i] == '마스크팩':
      hwahae_df.loc[i, '중분류1'] = '마스크팩'
      
    elif hwahae_df['중분류2'][i] == '기타':
      hwahae_df.loc[i, '중분류1'] = '기타'
      
    elif hwahae_df['중분류2'][i] == '선케어':
      hwahae_df.loc[i, '중분류1'] = '선케어'
    
    elif hwahae_df['중분류2'][i] == '립메이크업':
      hwahae_df.loc[i, '중분류1'] = '립메이크업'

In [180]:
print(hwahae_df['중분류1'].unique())
print(hwahae_df['중분류1'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '기타' '클렌징' '베이스메이크업' '마스크팩' '선케어' '아이메이크업' '립메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
...,...,...,...,...,...
4776,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4777,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4778,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4779,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업


In [181]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = hwahae_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = hwahae_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = hwahae_df['중분류1'].str.contains(colorclass)

for i in range(len(hwahae_df)):
  if cond_skinclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '색조메이크업'

In [182]:
# 대분류 고유값 확인 
print(hwahae_df['대분류'].unique())
print(hwahae_df['대분류'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
...,...,...,...,...,...
4776,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4777,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4778,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4779,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업


### 립메이크업 중분류 세분화

In [183]:
# musinsa_df
hwahae_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,URL,중분류1,중분류2
0,화해,에스네이처,아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜),1,스킨케어,스킨토너,NaN,4.73,17212,18900.0,NaN,스킨케어,토너/토너패드/스킨
1,화해,토리든,다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml),2,스킨케어,스킨토너,NaN,4.69,25849,19900.0,NaN,스킨케어,토너/토너패드/스킨
2,화해,라운드랩,1025 독도 토너 200ml + 독도 토너 500ml,3,스킨케어,스킨토너,NaN,4.43,85540,36000.0,NaN,스킨케어,토너/토너패드/스킨
3,화해,코스노리,판테놀 베리어 토너 205ml (2개입),4,스킨케어,스킨토너,NaN,4.76,2542,21900.0,NaN,스킨케어,토너/토너패드/스킨
4,화해,이즈앤트리,초저분자 히아루론산 토너 기획(300ml 본품+200ml 리필+미스트펌프),5,스킨케어,스킨토너,NaN,4.71,7026,15900.0,NaN,스킨케어,토너/토너패드/스킨


In [184]:
# 중분류2 복제
hwahae_df['중분류3'] = hwahae_df['중분류2']
hwahae_df['중분류3'].head()

0    토너/토너패드/스킨
1    토너/토너패드/스킨
2    토너/토너패드/스킨
3    토너/토너패드/스킨
4    토너/토너패드/스킨
Name: 중분류3, dtype: object

In [185]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = hwahae_df['중분류2'] == '립메이크업'
hwahae_df.loc[cond, '중분류'].unique()

array(['립스틱', '립틴트', '립글로스', '립케어_립밤', '컬러립케어_립밤'], dtype=object)

In [186]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

lipbalm_list = ['립밤']
lipbalm = '|'.join(lipbalm_list)
cond_lipbalm = hwahae_df['중분류'].str.contains(lipbalm)

for i in range(len(hwahae_df)):
  if cond_lipbalm[i] == True:
   hwahae_df.loc[i, '중분류3'] = '립밤'
  
  else:
    if hwahae_df['중분류'][i] == '립글로스':
      hwahae_df.loc[i, '중분류3'] = '립글로스'
      
    elif hwahae_df['중분류'][i] == '립스틱':
      hwahae_df.loc[i, '중분류3'] = '립스틱'
      
    elif hwahae_df['중분류'][i] == '립라이너':
      hwahae_df.loc[i, '중분류3'] = '립라이너'

    elif hwahae_df['중분류'][i] == '립틴트':
      hwahae_df.loc[i, '중분류3'] = '립틴트'


# 중분류3 입력값 확인
cond = hwahae_df['중분류2'] == '립메이크업'
print(hwahae_df.loc[cond, '중분류3'].unique())
print(hwahae_df.loc[cond, '중분류3'].nunique())
hwahae_df.loc[cond, ['중분류', '중분류3']].head()

['립스틱' '립틴트' '립글로스' '립밤']
4


,중분류,중분류3
4323,립스틱,립스틱
4324,립스틱,립스틱
4325,립스틱,립스틱
4326,립스틱,립스틱
4327,립스틱,립스틱


### 데이터셋 정리

In [187]:
# 기존 카테고리 지우기
hwahae_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
hwahae_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   평점      4781 non-null   float64
 6   후기 수    4781 non-null   int64  
 7   가격      4686 non-null   float64
 8   URL     0 non-null      float64
 9   중분류1    4781 non-null   object 
 10  중분류2    4781 non-null   object 
 11  중분류3    4781 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 448.3+ KB


In [188]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = hwahae_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = hwahae_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
hwahae_df = hwahae_df[reorder]

hwahae_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', 'URL'],
      dtype='object')

In [189]:
# 최종 데이터셋 확인
print(hwahae_df.info())
hwahae_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   중분류1    4781 non-null   object 
 6   중분류2    4781 non-null   object 
 7   중분류3    4781 non-null   object 
 8   평점      4781 non-null   float64
 9   후기 수    4781 non-null   int64  
 10  가격      4686 non-null   float64
 11  URL     0 non-null      float64
dtypes: float64(3), int64(2), object(7)
memory usage: 448.3+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,URL
0,화해,에스네이처,아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜),1,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.73,17212,18900.0,NaN
1,화해,토리든,다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml),2,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.69,25849,19900.0,NaN
2,화해,라운드랩,1025 독도 토너 200ml + 독도 토너 500ml,3,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.43,85540,36000.0,NaN
3,화해,코스노리,판테놀 베리어 토너 205ml (2개입),4,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.76,2542,21900.0,NaN
4,화해,이즈앤트리,초저분자 히아루론산 토너 기획(300ml 본품+200ml 리필+미스트펌프),5,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.71,7026,15900.0,NaN


In [190]:
# 최종 데이터셋 저장
# hwahae_df.to_csv('hwahae_preprocessing.csv', index=False, encoding="utf-8-sig")

In [192]:
hwahae_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   중분류1    4781 non-null   object 
 6   중분류2    4781 non-null   object 
 7   중분류3    4781 non-null   object 
 8   평점      4781 non-null   float64
 9   후기 수    4781 non-null   int64  
 10  가격      4686 non-null   float64
 11  URL     0 non-null      float64
dtypes: float64(3), int64(2), object(7)
memory usage: 448.3+ KB


## 올리브영

In [193]:
# 데이터셋 복제
oliveyoung_df = oliveyoung.copy(deep=True)

In [194]:
# 데이터셋 기본 정보 조회
print(oliveyoung_df.info())
print(oliveyoung_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None
    플랫폼   브랜드                                                제품명  중분류_순위  \
0  올리브영  에스트라  에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...       1   
1  올리브영  에스트라            에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획       2   
2  올리브영  에스트라  에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...       3   

    대분류        중분류        소분류   평점   후기 수

### 기본 전처리

In [195]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# 원본 데이터셋에 전처리 되어있음

'''
for i in range(len(oliveyoung_df)):
  score = oliveyoung_df['평점'][i]
  review = oliveyoung_df['후기 수'][i]
  price = oliveyoung_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  oliveyoung_df.loc[i, '평점'] = score_result
  oliveyoung_df.loc[i, '후기 수'] = review_result
  oliveyoung_df.loc[i, '가격'] = price_result

print(oliveyoung_df.loc[:10, ['평점', '후기 수', '가격']])
'''

"\nfor i in range(len(oliveyoung_df)):\n  score = oliveyoung_df['평점'][i]\n  review = oliveyoung_df['후기 수'][i]\n  price = oliveyoung_df['가격'][i]\n  \n  score_result = score.replace('별점 없음', '0')\n  review_result = review.replace(',', '')\n  price_result = price.replace(',', '')\n  \n  oliveyoung_df.loc[i, '평점'] = score_result\n  oliveyoung_df.loc[i, '후기 수'] = review_result\n  oliveyoung_df.loc[i, '가격'] = price_result\n\nprint(oliveyoung_df.loc[:10, ['평점', '후기 수', '가격']])\n"

In [196]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경
# 원본 데이터셋에 전처리 되어있음 

'''
oliveyoung_df[['평점', '후기 수', '가격']] = oliveyoung_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(oliveyoung_df[['평점', '후기 수', '가격']].dtypes)
print(oliveyoung_df.loc[:5, ['평점', '후기 수', '가격']])
'''

"\noliveyoung_df[['평점', '후기 수', '가격']] = oliveyoung_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)\n\nprint(oliveyoung_df[['평점', '후기 수', '가격']].dtypes)\nprint(oliveyoung_df.loc[:5, ['평점', '후기 수', '가격']])\n"

In [197]:
# 2. 컬럼 이름 변경

oliveyoung_df.rename(columns = {'정가':'가격'}, inplace=True)
oliveyoung_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '가격',
       '할인가'],
      dtype='object')

In [198]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

'''
oliveyoung_df['플랫폼'] = '올리브영'

a = oliveyoung_df.columns[-1:].to_list()

ex_a = oliveyoung_df.drop(columns = oliveyoung_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
oliveyoung_df = oliveyoung_df[reorder]

oliveyoung_df.columns
'''

"\noliveyoung_df['플랫폼'] = '올리브영'\n\na = oliveyoung_df.columns[-1:].to_list()\n\nex_a = oliveyoung_df.drop(columns = oliveyoung_df.columns[[-1]]).columns[:].to_list()\n\nreorder = a + ex_a\noliveyoung_df = oliveyoung_df[reorder]\n\noliveyoung_df.columns\n"

### 카테고리 전처리

In [199]:
# 1. 중분류1 & 중분류2 컬럼 생성
oliveyoung_df['중분류1'] = ''
print(oliveyoung_df['중분류1'].head())

oliveyoung_df['중분류2'] = ''
print(oliveyoung_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [200]:
# 기존 분류별 unique 값 확인
print(oliveyoung_df['대분류'].unique())
print(oliveyoung_df['중분류'].unique())
print(oliveyoung_df['소분류'].unique())

['스킨케어' '클렌징' '마스크팩' '선케어' '메이크업']
['크림' '에센스/세럼/앰플' '로션' '미스트/오일' '스킨/토너' '스킨케어세트' '클렌징폼/젤' '오일/밤' '워터/밀크'
 '필링&스크럽' '립&아이리무버' '티슈/패드' '패드' '시트팩' '페이셜팩' '패치' '코팩' '선크림' '선쿠션'
 '태닝/애프터선' '선스틱' '선스프레이/선패치' '베이스메이크업' '립메이크업' '아이메이크업']
['크림' '에센스/세럼/앰플' '로션' '미스트/픽서' '스킨/토너' '올인원' '페이스오일' '스킨케어세트' '클렌징폼/젤'
 '클렌징오일' '클렌징워터' '클렌징밤' '피지클리너' '팩클렌저' '페이셜스크럽' '클렌징밀크/크림' '립&아이리무버'
 '파우더워시' '클렌징티슈/패드' '패드' '시트팩' '워시오프팩' '패치' '필오프팩' '코팩' '슬리핑팩' '모델링팩'
 '선크림' '선쿠션' '애프터선' '선스틱' '선스프레이' '쿠션' '립스틱' '립틴트' '블러셔' '아이섀도우' '쉐이딩'
 '파운데이션' 'BB/CC' '아이라이너' '하이라이터' '마스카라' '아이브로우' '파우더/팩트' '컨실러' '프라이머/베이스'
 '립글로스' '립밤']


In [201]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = oliveyoung_df['중분류'] == '미스트/오일'
print(oliveyoung_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = oliveyoung_df['중분류'] == '스킨케어세트'
print(oliveyoung_df.loc[cond2, '대분류'].unique()) # 스킨케어

cond2 = oliveyoung_df['중분류'] == '패드'
print(oliveyoung_df.loc[cond2, '대분류'].unique()) # 마스크팩

['스킨케어']
['스킨케어']
['마스크팩']


In [202]:
# 패드 상품명 구체적으로 확인 : 토너 패드로 간주하고 대분류 자체를 스킨케어로 변경

cond = oliveyoung_df['중분류'] == '패드'
oliveyoung_df.loc[cond, ['대분류', '제품명']]

,대분류,제품명
200,마스크팩,더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획
201,마스크팩,더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획
202,마스크팩,더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트
203,마스크팩,메디힐 네모패드 100+100매 더블기획 7종 골라담기
206,마스크팩,메디힐 네모패드 100매 피부 고민별 골라담기
208,마스크팩,스킨푸드 캐롯 카로틴 카밍 워터 패드 60매+30매 한정기획 (+10매+캐롯시트 1매)
211,마스크팩,한율 5분 잡티톤업패드/트러블진정패드/모공청소패드 (달빛유자/어린쑥/부들밤)
212,마스크팩,메디큐브 제로모공패드 2.0 140매 기획 (70매 + 70매 리필)
214,마스크팩,넘버즈인 토너패드 리필기획 3종 중 택 1 (1번/4번/5번)
222,마스크팩,스킨푸드 캐롯 카로틴 카밍 워터 패드 기획(60매+30매 증정)


In [203]:
# 올인원 중분류 카테고리 확인 
cond = oliveyoung_df['소분류'] == '올인원' 
oliveyoung_df.loc[cond, '중분류'].unique() # 로션 or 기타 -> 세분카테고리 추가 의논 필요

array(['로션'], dtype=object)

In [204]:
# 통합 필요한 세부 카테고리 확인
cond1 = oliveyoung_df['대분류'] == '스킨케어'
print(oliveyoung_df.loc[cond1, '중분류'].unique())

cond2 = oliveyoung_df['중분류'] == '베이스메이크업'
print(oliveyoung_df.loc[cond2, '소분류'].unique())

cond3 = oliveyoung_df['중분류'] == '아이메이크업'
print(oliveyoung_df.loc[cond3, '소분류'].unique())

cond4 = oliveyoung_df['대분류'] == '마스크팩'
print(oliveyoung_df.loc[cond4, '중분류'].unique())

['크림' '에센스/세럼/앰플' '로션' '미스트/오일' '스킨/토너' '스킨케어세트']
['쿠션' '블러셔' '쉐이딩' '파운데이션' 'BB/CC' '하이라이터' '파우더/팩트' '컨실러' '프라이머/베이스']
['아이섀도우' '아이라이너' '마스카라' '아이브로우']
['패드' '시트팩' '페이셜팩' '패치' '코팩']


In [205]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['스킨', '토너']
skin = '|'.join(skin_list)
cond_skin = oliveyoung_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = oliveyoung_df['소분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '픽서']
fixer = '|'.join(fixer_list)
cond_fixer = oliveyoung_df['소분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = oliveyoung_df['소분류'].str.contains(shading)

primer_list = ['프라이머', '베이스', '톤업크림'] # 프라이머/베이스 컬럼에 (소분류)톤업크림 추가
primer = '|'.join(primer_list)
cond_primer = oliveyoung_df['소분류'].str.contains(primer)

for i in range(len(oliveyoung_df)):
  if cond_skin[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    oliveyoung_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if oliveyoung_df['중분류'][i] == '패드': # 패드를 검색 단어에 포함시키면 '(소분류)클렌징/패드'도 스킨케어에 포함될 수 있음
      oliveyoung_df.loc[i, '중분류2'] = '토너/토너패드/스킨' 
    
    elif oliveyoung_df['중분류'][i] == '로션':
      oliveyoung_df.loc[i, '중분류2'] = '로션'

    elif oliveyoung_df['중분류'][i] == '에센스/세럼/앰플':
      oliveyoung_df.loc[i, '중분류2'] = '에센스/세럼/앰플'
    
    elif oliveyoung_df['중분류'][i] == '크림':
      oliveyoung_df.loc[i, '중분류2'] = '크림'

    elif oliveyoung_df['대분류'][i] == '마스크팩':
      oliveyoung_df.loc[i, '중분류2'] = '마스크팩'

    elif oliveyoung_df['대분류'][i] == '클렌징':
      oliveyoung_df.loc[i, '중분류2'] = '클렌징'

    elif oliveyoung_df['대분류'][i] == '선케어':
      oliveyoung_df.loc[i, '중분류2'] = '선케어'

    elif oliveyoung_df['중분류'][i] == '립메이크업':
      oliveyoung_df.loc[i, '중분류2'] = '립메이크업'
    
    elif oliveyoung_df['소분류'][i] == '컨실러':
      oliveyoung_df.loc[i, '중분류2'] = '컨실러'
      
    elif oliveyoung_df['소분류'][i] == '아이라이너':
      oliveyoung_df.loc[i, '중분류2'] = '아이라이너'

    elif oliveyoung_df['소분류'][i] == '아이섀도우':
      oliveyoung_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif oliveyoung_df['소분류'][i] == '아이브로우':
      oliveyoung_df.loc[i, '중분류2'] = '아이브로우'

    elif oliveyoung_df['소분류'][i] == '마스카라':
      oliveyoung_df.loc[i, '중분류2'] = '마스카라'
    
    elif oliveyoung_df['소분류'][i] == '속눈썹영양제':
      oliveyoung_df.loc[i, '중분류2'] = '속눈썹영양제'
      
    else: # 기획세트
      oliveyoung_df.loc[i, '중분류2'] = '기타'

In [206]:
print(oliveyoung_df['중분류2'].unique())
print(oliveyoung_df['중분류2'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['크림' '에센스/세럼/앰플' '로션' '파우더/팩트/픽서' '토너/토너패드/스킨' '기타' '클렌징' '마스크팩' '선케어'
 '파운데이션/쿠션/BB/CC' '립메이크업' '블러셔/쉐이딩/하이라이터' '아이섀도우' '아이라이너' '마스카라' '아이브로우'
 '컨실러' '프라이머/베이스']
18


,대분류,중분류,소분류,중분류2
0,스킨케어,크림,크림,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
2,스킨케어,로션,로션,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,에센스/세럼/앰플
...,...,...,...,...
495,메이크업,립메이크업,립틴트,립메이크업
496,메이크업,아이메이크업,아이브로우,아이브로우
497,메이크업,아이메이크업,마스카라,마스카라
498,메이크업,립메이크업,립틴트,립메이크업


In [207]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = oliveyoung_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = oliveyoung_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = oliveyoung_df['중분류2'].str.contains(eye)

for i in range(len(oliveyoung_df)):
  if cond_skincare[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    oliveyoung_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if oliveyoung_df['중분류2'][i] == '클렌징':
      oliveyoung_df.loc[i, '중분류1'] = '클렌징'

    elif oliveyoung_df['중분류2'][i] == '마스크팩':
      oliveyoung_df.loc[i, '중분류1'] = '마스크팩'
      
    elif oliveyoung_df['중분류2'][i] == '기타':
      oliveyoung_df.loc[i, '중분류1'] = '기타'
      
    elif oliveyoung_df['중분류2'][i] == '선케어':
      oliveyoung_df.loc[i, '중분류1'] = '선케어'
    
    elif oliveyoung_df['중분류2'][i] == '립메이크업':
      oliveyoung_df.loc[i, '중분류1'] = '립메이크업'

In [208]:
print(oliveyoung_df['중분류1'].unique())
print(oliveyoung_df['중분류1'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '기타' '클렌징' '마스크팩' '선케어' '립메이크업' '아이메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,크림,크림,스킨케어,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
2,스킨케어,로션,로션,스킨케어,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
...,...,...,...,...,...
495,메이크업,립메이크업,립틴트,립메이크업,립메이크업
496,메이크업,아이메이크업,아이브로우,아이메이크업,아이브로우
497,메이크업,아이메이크업,마스카라,아이메이크업,마스카라
498,메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [209]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = oliveyoung_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = oliveyoung_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = oliveyoung_df['중분류1'].str.contains(colorclass)

for i in range(len(oliveyoung_df)):
  if cond_skinclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    oliveyoung_df.loc[i, '대분류'] = '색조메이크업'

In [210]:
# 대분류 고유값 확인 
print(oliveyoung_df['대분류'].unique())
print(oliveyoung_df['대분류'].nunique())
oliveyoung_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,크림,크림,스킨케어,크림
1,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
2,스킨케어,로션,로션,스킨케어,로션
3,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,스킨케어,에센스/세럼/앰플
...,...,...,...,...,...
495,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업
496,색조메이크업,아이메이크업,아이브로우,아이메이크업,아이브로우
497,색조메이크업,아이메이크업,마스카라,아이메이크업,마스카라
498,색조메이크업,립메이크업,립틴트,립메이크업,립메이크업


In [211]:
# '(중분류)패드'의 분류값 확인
cond = oliveyoung_df['중분류'] == '패드'
oliveyoung_df.loc[cond, ['대분류', '중분류1', '중분류2']]

,대분류,중분류1,중분류2
200,스킨케어,스킨케어,토너/토너패드/스킨
201,스킨케어,스킨케어,토너/토너패드/스킨
202,스킨케어,스킨케어,토너/토너패드/스킨
203,스킨케어,스킨케어,토너/토너패드/스킨
206,스킨케어,스킨케어,토너/토너패드/스킨
208,스킨케어,스킨케어,토너/토너패드/스킨
211,스킨케어,스킨케어,토너/토너패드/스킨
212,스킨케어,스킨케어,토너/토너패드/스킨
214,스킨케어,스킨케어,토너/토너패드/스킨
222,스킨케어,스킨케어,토너/토너패드/스킨


### 립메이크업 중분류 세분화

In [212]:
# musinsa_df
oliveyoung_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,할인가,중분류1,중분류2
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,크림,크림,4.9,25321,59400,44500.0,스킨케어,크림
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000,24700.0,스킨케어,에센스/세럼/앰플
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,로션,로션,4.9,2688,53400,39900.0,스킨케어,로션
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500,27300.0,스킨케어,에센스/세럼/앰플
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000,32900.0,스킨케어,에센스/세럼/앰플


In [213]:
# 중분류2 복제
oliveyoung_df['중분류3'] = oliveyoung_df['중분류2']
oliveyoung_df['중분류3'].head()

0           크림
1    에센스/세럼/앰플
2           로션
3    에센스/세럼/앰플
4    에센스/세럼/앰플
Name: 중분류3, dtype: object

In [214]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = oliveyoung_df['중분류2'] == '립메이크업'
oliveyoung_df.loc[cond, '소분류'].unique()

array(['립스틱', '립틴트', '립글로스', '립밤'], dtype=object)

In [215]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

for i in range(len(oliveyoung_df)):
  if oliveyoung_df['소분류'][i] == '립글로스':
    oliveyoung_df.loc[i, '중분류3'] = '립글로스'

  elif oliveyoung_df['소분류'][i] == '립밤':
    oliveyoung_df.loc[i, '중분류3'] = '립밤'
    
  elif oliveyoung_df['소분류'][i] == '립스틱':
    oliveyoung_df.loc[i, '중분류3'] = '립스틱'
    
  elif oliveyoung_df['소분류'][i] == '립라이너':
    oliveyoung_df.loc[i, '중분류3'] = '립라이너'

  elif oliveyoung_df['소분류'][i] == '립틴트':
    oliveyoung_df.loc[i, '중분류3'] = '립틴트'


# 중분류3 입력값 확인
cond = oliveyoung_df['중분류2'] == '립메이크업'
print(oliveyoung_df.loc[cond, '중분류3'].unique())
print(oliveyoung_df.loc[cond, '중분류3'].nunique())
oliveyoung_df.loc[cond, ['소분류', '중분류3']].head()

['립스틱' '립틴트' '립글로스' '립밤']
4


,소분류,중분류3
401,립스틱,립스틱
402,립틴트,립틴트
404,립틴트,립틴트
408,립틴트,립틴트
409,립틴트,립틴트


### 데이터셋 정리

In [216]:
# 기존 카테고리 지우기
oliveyoung_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
oliveyoung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   평점      500 non-null    float64
 6   후기 수    500 non-null    int64  
 7   가격      500 non-null    int64  
 8   할인가     432 non-null    float64
 9   중분류1    500 non-null    object 
 10  중분류2    500 non-null    object 
 11  중분류3    500 non-null    object 
dtypes: float64(2), int64(3), object(7)
memory usage: 47.0+ KB


In [217]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = oliveyoung_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = oliveyoung_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
oliveyoung_df = oliveyoung_df[reorder]

oliveyoung_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', '할인가'],
      dtype='object')

In [218]:
# 최종 데이터셋 확인
print(oliveyoung_df.info())
oliveyoung_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류1    500 non-null    object 
 6   중분류2    500 non-null    object 
 7   중분류3    500 non-null    object 
 8   평점      500 non-null    float64
 9   후기 수    500 non-null    int64  
 10  가격      500 non-null    int64  
 11  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(7)
memory usage: 47.0+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,할인가
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400,44500.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000,24700.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400,39900.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500,27300.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000,32900.0


In [219]:
# 최종 데이터셋 저장
# oliveyoung_df.to_csv('oliveyoung_preprocessing.csv', index=False, encoding="utf-8-sig")

## 플랫폼 데이터 통합

### 데이터셋 통합

In [123]:
# 플랫폼 데이터셋별 컬럼 정보 확인
print(musinsa_df.info()) # 12개 컬럼 - 할인가 X, URL O
print(hwahae_df.info()) # 12개 컬럼 - 할인가 X, URL O
print(oliveyoung_df.info()) # 12개 컬럼 - 할인가 O, URL X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   중분류1    400 non-null    object 
 6   중분류2    400 non-null    object 
 7   중분류3    400 non-null    object 
 8   평점      400 non-null    float64
 9   후기 수    400 non-null    int64  
 10  가격      400 non-null    int64  
 11  URL     400 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781

In [124]:
# 컬럼 맞추기
musinsa_df2 = musinsa_df.drop(columns={'URL'})
print(musinsa_df2.info())

hwahae_df2 = hwahae_df.drop(columns={'URL'})
print(hwahae_df2.info())

oliveyoung_df2 = oliveyoung_df.drop(columns={'할인가'})
print(oliveyoung_df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     400 non-null    object 
 1   브랜드     400 non-null    object 
 2   제품명     400 non-null    object 
 3   중분류_순위  400 non-null    int64  
 4   대분류     400 non-null    object 
 5   중분류1    400 non-null    object 
 6   중분류2    400 non-null    object 
 7   중분류3    400 non-null    object 
 8   평점      400 non-null    float64
 9   후기 수    400 non-null    int64  
 10  가격      400 non-null    int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 34.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781

In [125]:
korea_all = pd.concat([musinsa_df2, hwahae_df2, oliveyoung_df2], axis=0, ignore_index=True)
print(korea_all.info())
korea_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     5681 non-null   object 
 1   브랜드     5681 non-null   object 
 2   제품명     5681 non-null   object 
 3   중분류_순위  5681 non-null   int64  
 4   대분류     5681 non-null   object 
 5   중분류1    5681 non-null   object 
 6   중분류2    5681 non-null   object 
 7   중분류3    5681 non-null   object 
 8   평점      5681 non-null   float64
 9   후기 수    5681 non-null   int64  
 10  가격      5586 non-null   float64
dtypes: float64(2), int64(2), object(7)
memory usage: 488.3+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,무신사뷰티,이니스프리,화산송이 모공 바하 클렌징 폼 150g 2개,1,스킨케어,클렌징,클렌징,클렌징,4.9,7915,18200.0
1,무신사뷰티,비플레인,[대용량]녹두 약산성 클렌징 폼 160ml 2개 (+40ml 증정),2,스킨케어,클렌징,클렌징,클렌징,4.9,2870,31200.0
2,무신사뷰티,에스네이처,아쿠아 라이스 약산성 클렌징폼 160ml 2개입 [증정: 클렌징 오일 13ml + ...,3,스킨케어,클렌징,클렌징,클렌징,4.9,630,31900.0
3,무신사뷰티,토리든,[2개묶음] 다이브인 저분자 히알루론산 클렌징 폼 150ml x 2개 [총 300ml],4,스킨케어,클렌징,클렌징,클렌징,4.9,3707,23000.0
4,무신사뷰티,바닐라코,클린잇제로 오리지널 클렌징밤 대용량 180ml,5,스킨케어,클렌징,클렌징,클렌징,4.9,1215,23000.0


### 통합 데이터 전처리

In [126]:
# 1. 해외 브랜드 및 바디&헤어케어 브랜드 삭제
# 삭제할 브랜드명 리스트 
not_korean = ['에센허브', '다슈', '바이오더마', '오프라 코스메틱', '비오레', '키스미', '라로슈포제', '피지오겔', '비오템', '비오텀', '라씨엘르', '베네피트', '센카', '아크네스', '에스티로더', '유세린', '폰즈', '블리스텍스', '테', '아넷사', '스틸라', '그라펜', '무신사', '시코르', '뉴트로지나', '시슬리', '아벤느', '디올', '크리니크', '이솝', '샤넬', '바비브라운', '록시땅', '꼬달리', '프레쉬', '라로제', '폴메디슨', '올리세', '유리아쥬', '히루스카', '나스', '발레아', '큐어시스', '맥', '슈에무라', '비오투름', '에스테덤', '산수시', '노레바', '러쉬', '소티스', '이브롬', '마리꼬', '스킨레지민', '도브', '씨드비', '랑팔라투르', '달란디올리브', '바이오티크', '다이알', '쌍빠', '라부르켓', '블리스텍스', '후로후시', '딸리까', '제이숲', '에스까다코스메틱', '바이브랩', '케이트', '레브론', '존슨즈베이비', '샬롯틸버리', '토브', '샤넬', '클라랑스', '메이크업포에버', '지방시', '샹테카이', '로라메르시에', '입생로랑', '로레알', '아르마니', '메이블린', '안나수이', '투페이스드', '어반디케이']
nk = '|'.join(not_korean)

# 리스트 중 하나라도 포함되면 전체 삭제 
cond_nk = korea_all['브랜드'].str.contains(nk)
nk_idx = korea_all[cond_nk == True].index

korea_all.drop(nk_idx, inplace=True)

# 인덱스 재정렬
korea_all.reset_index(drop=True, inplace=True)
korea_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170 entries, 0 to 5169
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     5170 non-null   object 
 1   브랜드     5170 non-null   object 
 2   제품명     5170 non-null   object 
 3   중분류_순위  5170 non-null   int64  
 4   대분류     5170 non-null   object 
 5   중분류1    5170 non-null   object 
 6   중분류2    5170 non-null   object 
 7   중분류3    5170 non-null   object 
 8   평점      5170 non-null   float64
 9   후기 수    5170 non-null   int64  
 10  가격      5099 non-null   float64
dtypes: float64(2), int64(2), object(7)
memory usage: 444.4+ KB


In [130]:
korea_all[korea_all['플랫폼']=='올리브영']

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
4716,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
4717,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
4718,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
4719,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0
4720,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0
...,...,...,...,...,...,...,...,...,...,...,...
5165,올리브영,더샘,더샘 커버 퍼펙션 팁컨실러 9.5g 6colors,95,베이스메이크업,베이스메이크업,컨실러,컨실러,4.7,15210,7700.0
5166,올리브영,롬앤,롬앤 듀이풀 워터 틴트,96,색조메이크업,립메이크업,립메이크업,립틴트,4.7,36721,13000.0
5167,올리브영,롬앤,롬앤 한올 샤프 브로우,97,색조메이크업,아이메이크업,아이브로우,아이브로우,4.7,14002,12000.0
5168,올리브영,클리오,클리오 크리스탈 글램 틴트,99,색조메이크업,립메이크업,립메이크업,립틴트,4.7,21536,18000.0


In [131]:
# 2. 순위 재정립
# 플랫폼 - 중분류1 단위별 reset_index -중분류_순위 = index+1
# df["새로운_순위"] = df.groupby(["플랫폼", "중분류1"])["중분류_순위"].rank(method="first", ascending=True).astype(int)

korea_all['new_rank'] = korea_all.groupby(['플랫폼', '중분류1'])['중분류_순위'].rank(method='dense', ascending=True).astype(int)

In [132]:
korea_all[korea_all['플랫폼']=='올리브영']

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,new_rank
4716,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0,1
4717,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0,2
4718,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0,3
4719,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0,4
4720,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
5165,올리브영,더샘,더샘 커버 퍼펙션 팁컨실러 9.5g 6colors,95,베이스메이크업,베이스메이크업,컨실러,컨실러,4.7,15210,7700.0,51
5166,올리브영,롬앤,롬앤 듀이풀 워터 틴트,96,색조메이크업,립메이크업,립메이크업,립틴트,4.7,36721,13000.0,19
5167,올리브영,롬앤,롬앤 한올 샤프 브로우,97,색조메이크업,아이메이크업,아이브로우,아이브로우,4.7,14002,12000.0,28
5168,올리브영,클리오,클리오 크리스탈 글램 틴트,99,색조메이크업,립메이크업,립메이크업,립틴트,4.7,21536,18000.0,20


In [133]:
# 3. 인덱스 정렬 및 컬럼 정리
# 인덱스 정렬
korea_all.reset_index(drop=True, inplace=True)

# 기존 '중분류_순위' 값을 'new_rank' 값으로 바꾸기
korea_all['중분류_순위'] = korea_all['new_rank']

# 'new_rank' 지우기
korea_all.drop(columns = {'new_rank'}, axis=1, inplace=True)

print(korea_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170 entries, 0 to 5169
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     5170 non-null   object 
 1   브랜드     5170 non-null   object 
 2   제품명     5170 non-null   object 
 3   중분류_순위  5170 non-null   int32  
 4   대분류     5170 non-null   object 
 5   중분류1    5170 non-null   object 
 6   중분류2    5170 non-null   object 
 7   중분류3    5170 non-null   object 
 8   평점      5170 non-null   float64
 9   후기 수    5170 non-null   int64  
 10  가격      5099 non-null   float64
dtypes: float64(2), int32(1), int64(1), object(7)
memory usage: 424.2+ KB
None


In [138]:
olive_check = korea_all[korea_all['플랫폼'] == '올리브영']
olive_check[olive_check['중분류1']=='스킨케어']

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
4716,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 ...,1,스킨케어,스킨케어,크림,크림,4.9,25321,59400.0
4717,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000.0
4718,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,스킨케어,로션,로션,4.9,2688,53400.0
4719,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500.0
4720,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000.0
...,...,...,...,...,...,...,...,...,...,...,...
4964,올리브영,바이오던스,바이오던스 겔 토너 패드 60매 (씨켈프/비타),76,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.9,46,26000.0
4965,올리브영,에스네이처,에스네이처 아쿠아 오아시스 판테알란 카밍패드 60매,77,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.9,1749,26000.0
4968,올리브영,아누아,아누아 비타민씨 나이아신 톤업 잡티 패드 60매,80,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,5.0,49,24000.0
4970,올리브영,에스트라,에스트라 에이시카365 쿨링팩패드 pH4.5 60매,82,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,5.0,5,28000.0


In [140]:
hwahae_check = korea_all[korea_all['플랫폼'] == '화해']
hwahae_check[hwahae_check['중분류1']=='스킨케어']

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
370,화해,에스네이처,아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜),1,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.73,17212,18900.0
371,화해,토리든,다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml),2,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.69,25849,19900.0
372,화해,라운드랩,1025 독도 토너 200ml + 독도 토너 500ml,3,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.43,85540,36000.0
373,화해,코스노리,판테놀 베리어 토너 205ml (2개입),4,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.76,2542,21900.0
374,화해,이즈앤트리,초저분자 히아루론산 토너 기획(300ml 본품+200ml 리필+미스트펌프),5,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.71,7026,15900.0
...,...,...,...,...,...,...,...,...,...,...,...
2775,화해,토코보,시카 카밍 아쿠아 패드 60매,96,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.90,10,21600.0
2776,화해,오어스,순한 진정 순진 비건 토너패드 90매,97,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.53,150,20500.0
2777,화해,네이처리퍼블릭,굿 스킨 티트리 앰플 토너 패드 (50매) 125g,98,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.66,79,6600.0
2778,화해,믹순,두유패드 10개입,99,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.60,98,19800.0


### 데이터셋 저장

In [ ]:
# 통합 파일 저장
# korea_all.to_csv('korea_all_preprocessing.csv', index=False, encoding="utf-8-sig")

In [136]:
# 파일 저장본 확인인
# korea_check = pd.read_csv('korea_all_preprocessing.csv')
# korea_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170 entries, 0 to 5169
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     5170 non-null   object 
 1   브랜드     5170 non-null   object 
 2   제품명     5170 non-null   object 
 3   중분류_순위  5170 non-null   int64  
 4   대분류     5170 non-null   object 
 5   중분류1    5170 non-null   object 
 6   중분류2    5170 non-null   object 
 7   중분류3    5170 non-null   object 
 8   평점      5170 non-null   float64
 9   후기 수    5170 non-null   int64  
 10  가격      5099 non-null   float64
dtypes: float64(2), int64(2), object(7)
memory usage: 444.4+ KB


## 코스메

In [ ]:
# 데이터셋 복제
cosme_df = cosme.copy(deep=True)

In [ ]:
# 데이터셋 기본 정보 조회
print(cosme_df.info())
print(cosme_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   순위       3000 non-null   int64  
 1   상품ID     3000 non-null   int64  
 2   브랜드_KR   3000 non-null   object 
 3   제품명_EN   3000 non-null   object 
 4   카테고리_EN  3000 non-null   object 
 5   평점       3000 non-null   float64
 6   리뷰수_EN   3000 non-null   object 
 7   가격       3000 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 187.6+ KB
None
   순위      상품ID   브랜드_KR                    제품명_EN          카테고리_EN   평점  \
0   1  10189359  브이티코스메틱  CICA Daily Soothing Mask  Sheet mask pack  5.2   
1   2  10145207       더샘         CP Chip Concealer        Concealer  4.6   
2   3  10176465       롬앤        Juicy Lasting Tint         lipstick  4.9   

          리뷰수_EN    가격  
0  10034 reviews  2420  
1   8804 reviews   858  
2   6709 reviews  1320  


### 기본 전처리

In [ ]:
cosme_df[['평점','리뷰수_EN','가격']]

,평점,리뷰수_EN,가격
0,5.2,10034 reviews,2420
1,4.6,8804 reviews,858
2,4.9,6709 reviews,1320
3,4.8,"6,530 reviews",2970
4,5.2,"6,457 reviews",1090
...,...,...,...
2995,6.0,17 reviews,990
2996,5.1,17 reviews,1430
2997,5.0,17 reviews,1980
2998,4.9,17 reviews,1320


In [ ]:
max_len = cosme_df['리뷰수_EN'].str.len().max()

cosme_df[cosme_df['리뷰수_EN'].str.len()==max_len]

,순위,상품ID,브랜드_KR,제품명_EN,카테고리_EN,평점,리뷰수_EN,가격
0,1,10189359,브이티코스메틱,CICA Daily Soothing Mask,Sheet mask pack,5.2,10034 reviews,2420
3,4,10217471,티르티르,MASK FIT RED CUSHION,Liquid foundation,4.8,"6,530 reviews",2970
4,5,10245780,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,Sheet mask pack,5.2,"6,457 reviews",1090
6,7,10217712,이니스프리,Retinol Deer Repair Serum,Beauty fluid,4.7,"5,574 reviews",3960
7,8,10198590,아누아,Dokudami 77 Soothing Toner,Lotion,5.0,"5,308 reviews",2380
9,10,10216004,넘버즈인,No. 3 Smooth care sheet mask,Sheet mask pack,5.0,"4,383 reviews",1090
11,12,10137755,메디힐,N.M.F Aqua Ampoule Mask JEX,Sheet mask pack,5.4,"4,131 reviews",356
15,16,2926413,미샤,BB Cream UV,Sunscreen and UV care (for face),4.7,"3,420 reviews",2626
23,24,10225136,코스알엑스,RX the Vitamin C23 Serum,Beauty fluid,5.0,"2,238 reviews",2480
24,25,10235552,Wonjungyo,Won Jeongyo Metal Shower Pencil,Gel and cream eyeshadow,5.2,"2,178 reviews",1650


In [ ]:
# 1. 컬럼 타입 변경 : 리뷰수_EN(int)
# 1) 리뷰수_EN 컬럼의 ' reviews', ',', '??' 삭제

for i in range(len(cosme_df)):
  review = cosme_df['리뷰수_EN'][i]

  review_result = review.replace('reviews', '')
  review_result = review_result.replace(',', '')
  review_result = review_result.replace(' ', '')
  review_result = review_result.replace('??', '')
  
  cosme_df.loc[i, '리뷰수_EN'] = review_result

print(cosme_df.loc[:10, '리뷰수_EN'])

0     10034
1      8804
2      6709
3      6530
4      6457
5      5865
6      5574
7      5308
8      4704
9      4383
10     4267
Name: 리뷰수_EN, dtype: object


In [ ]:
# 1. 컬럼 타입 변경 : 리뷰수_EN(int)
# 2) 컬럼 타입 변경

cosme_df['리뷰수_EN'] = cosme_df['리뷰수_EN'].apply(pd.to_numeric)

print(cosme_df['리뷰수_EN'].dtypes)
print(cosme_df.loc[:5, '리뷰수_EN'])

int64
0    10034
1     8804
2     6709
3     6530
4     6457
5     5865
Name: 리뷰수_EN, dtype: int64


In [ ]:
# 2. 컬럼 이름 변경

cosme_df.columns = ['순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격']
cosme_df.columns

Index(['순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격'], dtype='object')

In [ ]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

cosme_df['플랫폼'] = '코스메' 

a = cosme_df.columns[-1:].to_list()

ex_a = cosme_df.drop(columns = cosme_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
cosme_df = cosme_df[reorder]

cosme_df.columns

Index(['플랫폼', '순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격'], dtype='object')

### 대분류 및 중분류 통합

In [ ]:
# 1. 중분류1 & 중분류3 컬럼 생성
cosme_df['중분류1'] = ''
print(cosme_df['중분류1'].head())

cosme_df['중분류3'] = ''
print(cosme_df['중분류3'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류3, dtype: object


In [ ]:
# 기존 분류 unique 값 확인
print(cosme_df['소분류'].unique()) # -> 제품명에 포함된 이름으로 찾기

['Sheet mask pack' 'Concealer' 'lipstick' 'Liquid foundation'
 'Brow mascara' 'Beauty fluid' 'Lotion' 'Loose powder' 'Face cream'
 'Sunscreen and UV care (for face)' 'Powder eyeshadow' 'mascara'
 'Lip care and lip balm' 'Washing packs and masks'
 'Gel and cream eyeshadow' 'Presto powder' 'Other facial cleansers'
 'Oil cleansing' 'Shampoo and conditioner' 'Mist-like lotion'
 'Booster/Introducing Fluid' 'Other cleansing' 'Lip gloss'
 'Eye Care Eye Cream' 'Other foundations' 'Other skin care'
 'Pencil eyeliner' 'Makeup base' 'Liquid eyeliner' 'Gomage Peeling'
 'Liquid Cleansing' 'Manicure' 'Skincare kit' 'Hair packs and treatments'
 'Powder Cheek' 'Out-of-bath treatment' 'Eyebrow pencil'
 'Gel and cream blush' 'Face wash foam' 'Milk' 'Puff sponge'
 'Makeup brush' 'False nails and nail tips'
 'Perfumes and Fragrances (Other)' 'Face Oil Balm' 'Eyelash beauty liquid'
 'Powder foundation' 'Cream and gel foundation' 'Body soap'
 'Body lotion/milk' 'Powder eyebrow' 'Other body care' 'Gel Eyelin

- 삭제할 소분류 검색값: 'pouch', 'brush', 'body', 'Body', 'hair', 'Hair', 'Shampoo', 'perfume', 'Hand', 'hand', 'bath', 'Other skin care products', 'nail', 'Nail', 'Health', 'food', 'drink', 'supplement', 'Other eyeliner', 'Shaving remover', 'Other makeup products', 'Manicure', 'Appliance', 'Leg', 'Foot', 'Travel', 'Other kit', 'Puff', 'cotton', 'Medical', 'Scalp', 'others'
- 삭제할 브랜드 : 'Mijangsen'(헤어케어브랜드), 'Stilla'(한국 브랜드 아님), 'Soon Plus'(단백질파우더 팔고 있음)

In [ ]:
# 불필요한 행 삭제하기
# 1) 소분류 값 검색

word_list = ['pouch', 'brush', 'body', 'Body', 'hair', 'Hair', 'Shampoo', 'shampoo', 'perfume', 'Perfumes', 'Hand', 'hand', 'bath', 'Other skin care products', 'nail', 'Nail', 'Health', 'food', 'Food', 'drink', 'Drink', 'supplement', 'Other eyeliner', 'Shaving remover', 'Other makeup products', 'Manicure', 'Appliance', 'Leg', 'Foot', 'Travel', 'Other kit', 'Puff', 'cotton', 'Medical', 'Scalp', 'others', 'Blotting']
word = '|'.join(word_list)

word_idx = cosme_df[cosme_df['소분류'].str.contains(word)].index

cosme_df = cosme_df.drop(word_idx)
cosme_df.info() # 전체행 3000개 -> 2671개로 감소

<class 'pandas.core.frame.DataFrame'>
Index: 2671 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2671 non-null   object 
 1   순위      2671 non-null   int64  
 2   제품ID    2671 non-null   int64  
 3   브랜드     2671 non-null   object 
 4   제품명     2671 non-null   object 
 5   소분류     2671 non-null   object 
 6   평점      2671 non-null   float64
 7   후기 수    2671 non-null   int64  
 8   가격      2671 non-null   int64  
 9   중분류1    2671 non-null   object 
 10  중분류3    2671 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 250.4+ KB


In [ ]:
# 불필요한 행 삭제하기
# 2) 불필요한 브랜드 삭제

brand_list = ['Mijangsen', 'Stilla', 'Soon Plus']
brand = '|'.join(brand_list)

brand_idx = cosme_df[cosme_df['브랜드'].str.contains(brand)].index

cosme_df = cosme_df.drop(brand_idx)
cosme_df.info() # 전체행 2673개 -> 2581개로 감소

<class 'pandas.core.frame.DataFrame'>
Index: 2581 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   순위      2581 non-null   int64  
 2   제품ID    2581 non-null   int64  
 3   브랜드     2581 non-null   object 
 4   제품명     2581 non-null   object 
 5   소분류     2581 non-null   object 
 6   평점      2581 non-null   float64
 7   후기 수    2581 non-null   int64  
 8   가격      2581 non-null   int64  
 9   중분류1    2581 non-null   object 
 10  중분류3    2581 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 242.0+ KB


In [ ]:
# 불필요한 행 삭제하기
# 3) 인덱스 리셋

cosme_df.reset_index(drop=True, inplace=True)
cosme_df.head()

,플랫폼,순위,제품ID,브랜드,제품명,소분류,평점,후기 수,가격,중분류1,중분류3
0,코스메,1,10189359,브이티코스메틱,CICA Daily Soothing Mask,Sheet mask pack,5.2,10034,2420,,
1,코스메,2,10145207,더샘,CP Chip Concealer,Concealer,4.6,8804,858,,
2,코스메,3,10176465,롬앤,Juicy Lasting Tint,lipstick,4.9,6709,1320,,
3,코스메,4,10217471,티르티르,MASK FIT RED CUSHION,Liquid foundation,4.8,6530,2970,,
4,코스메,5,10245780,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,Sheet mask pack,5.2,6457,1090,,


스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
1) 마스크팩: (소)'Sheet mask pack'
2) 클렌징: (소)'wash', (소)'Peeling', (소)'clean', (소)'Clean', (소)'Remover'

3) 토너/토너패드/스킨
    - (소)'Lotion' => 이름에 'toner', 'Toner', 'pad', 'Pad','water', 'Water', 'skin', 'Skin', 'Shigeakisui'(설화수 제품명), 'mask', 'Mask' 들어가는 경우

4) 에센스/세럼/앰플: (소)'Booster', (소)'Beauty fluid'
    - (소)'Lotion' => 이름에 'essence', 'Essence', 'Essener', 'serum', 'Serum', 'booster' 들어가는 경우

5) 로션: (소)'Milk'
    - (소)'Lotion' => 이름에 'lotion', 'Lotion' 들어가는 경우 & 1)~4)에서 필터링하고 남은 나머지 상품들들

6) 크림
    - (소)'Face cream' => 'BB' 제외 나머지 제품

7) 기타: (소)'mist', (소)'Oil Balm', (소)'Skincare kit', (소)'Other skin care', (소)'Eye Cream', (소)'All', (소)'Skincare kit'

베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
1) 선케어: (소)'Sunscreen'

2) 파운데이션/쿠션/BB/CC: (소)'foundation'
    - (소)'Face cream' => 이름에 'BB' 들어갈 경우

3) 파우더/팩트/픽서:
    - (소)'Cream and gel foundation'=> 이름에 'pact' 들어가는 경우 / 이외 모두 파운데이션으로 분류
    - (소) 'Presto powder' => 이름에 'powder', 'Powder', 'finsh', 'Finish', 'pact', 'Pact' 들어가는 경우
    - (소)'Loose'

4) 블러셔/쉐이딩/하이라이터: (소)'blush', (소)'Cheek'
    - (소) 'Presto powder' => 이름에 'shading' 'Shading', 'highlighter', 'Highlighter', 'blusher', 'Blusher', 'brusher', 'Brusher', 'contoure', 'Contoure' 들어가는 경우

5) 프라이머/베이스: (소)'Makeup base'

6) 컨실러: (소)'Concealer'

색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
1) 립메이크업
    - 립글로스: (소)'Lip gloss'
    - 립밤: (소)'lip balm'
    - 립스틱: (소)'lipstick'
    - 립라이너: (소)'Lip liner'
    - 립틴트: 립글로스, 립스틱 중에서 이름에 'tint' 포함된 경우

2) 아이섀도우: (소)'eyeshadow', (소)'Palettes'

3) 아이라이너: (소)'eyeliner'

4) 아이브로우: (소)'eyebrow', (소)'Eyebrow', (소)'Brow mascara'

5) 마스카라: (소)'mascara', (소)'Mascara'

6) 속눈썹 영양제: (소)'Eyelash'

In [ ]:
# 중분류3

clean_list = ['wash', 'Peeling', 'Wash', 'clean', 'Clean', 'Remover']
cleansing = '|'.join(clean_list)
cond_clean = cosme_df['소분류'].str.contains(cleansing)

toner_list = ['toner', 'Toner', 'pad', 'Pad','water', 'Water', 'skin', 'Skin', 'Shigeakisui', 'mask', 'Mask']
toner = '|'.join(toner_list)
cond_toner = cosme_df['제품명'].str.contains(toner)

serum_list = ['essence', 'Essence', 'Essener', 'serum', 'Serum', 'booster', 'Booster', 'Beauty fluid']
serum = '|'.join(serum_list)
cond_serum = cosme_df['제품명'].str.contains(serum)
cond_serum2 = cosme_df['소분류'].str.contains(serum)

extra_list = ['mist', 'Mist', 'Oil Balm', 'Skincare kit', 'Other skin care', 'Eye Cream', 'All', 'Skincare kit']
extra = '|'.join(extra_list)
cond_extra = cosme_df['소분류'].str.contains(extra)
#---------------------------------------------------------#
cond_bb = cosme_df['제품명'].str.contains('BB')

powder_list = ['powder', 'Powder', 'finsh', 'Finish', 'pact', 'Pact']
powder = '|'.join(powder_list)
cond_powder = cosme_df['제품명'].str.contains(powder)

shading_list = ['shading' 'Shading', 'highlighter', 'Highlighter', 'blusher', 'Blusher', 'brusher', 'Brusher', 'contoure', 'Contoure', 'face cube']
shading = '|'.join(shading_list)
cond_shading = cosme_df['제품명'].str.contains(shading)

blusher_list = ['blush', 'Cheek']
blusher = '|'.join(blusher_list)
cond_blusher = cosme_df['소분류'].str.contains(blusher)

cond_sun = cosme_df['소분류'].str.contains('Sunscreen')
cond_found = cosme_df['소분류'].str.contains('foundation')
cond_loose = cosme_df['소분류'].str.contains('Loose')
cond_base = cosme_df['소분류'].str.contains('Makeup base')
#---------------------------------------------------------#
cond_tint = cosme_df['제품명'].str.contains('tint')
cond_balm = cosme_df['소분류'].str.contains('lip balm')
cond_liner = cosme_df['소분류'].str.contains('Lip liner')
#---------------------------------------------------------#
shadow_list = ['eyeshadow', 'Palettes']
shadow = '|'.join(shadow_list)
cond_shadow = cosme_df['소분류'].str.contains(shadow)

brow_list = ['eyebrow', 'Eyebrow', 'Brow mascara']
brow= '|'.join(brow_list)
cond_brow = cosme_df['소분류'].str.contains(brow)

masc_list = ['mascara', 'Mascara']
masc= '|'.join(masc_list)
cond_masc = cosme_df['소분류'].str.contains(masc)

line_list = ['eyeliner', 'Eyeliner']
eyeline = '|'.join(line_list)
cond_eyeline = cosme_df['소분류'].str.contains(eyeline)

lash_list = ['eyelash', 'Eyelash']
eyelash = '|'.join(lash_list)
cond_eyelash = cosme_df['소분류'].str.contains(eyelash)
#---------------------------------------------------------#

for i in range(0, len(cosme_df)):
  if cosme_df['소분류'][i] == 'Lotion':
    if cond_toner[i] == True :
      cosme_df.loc[i, '중분류3'] = '토너/토너패드/스킨'
    elif cond_serum[i] == True:
      cosme_df.loc[i, '중분류3'] = '에센스/세럼/앰플'
    else:
      cosme_df.loc[i, '중분류3'] = '로션'
  
  elif cosme_df['소분류'][i] == 'Face cream':
    if cond_bb[i] == True:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'
    else:
      cosme_df.loc[i, '중분류3'] = '크림'

  elif cosme_df['소분류'][i] == 'Cream and gel foundation':
    if cond_powder[i] == True:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'
    else:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'
  
  elif cosme_df['소분류'][i] == 'Presto powder':
    if cond_shading[i] == True:
      cosme_df.loc[i, '중분류3'] = '블러셔/쉐이딩/하이라이터'
    else:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'      
  
  elif (cosme_df['소분류'][i]=='Lip gloss') | (cosme_df['소분류'][i]=='lipstick') :
    if cond_tint[i] == True:
      cosme_df.loc[i, '중분류3'] = '립틴트'
    elif cosme_df['소분류'][i] == 'Lip gloss':
      cosme_df.loc[i, '중분류3'] = '립글로스'      
    else:
      cosme_df.loc[i, '중분류3'] = '립스틱'
      
  else:

    if cond_clean[i] == True:
      cosme_df.loc[i, '중분류3'] = '클렌징'
    
    elif cosme_df['소분류'][i] == 'Sheet mask pack':
      cosme_df.loc[i, '중분류3'] = '마스크팩'

    elif cond_serum2[i] == True:
      cosme_df.loc[i, '중분류3'] = '에센스/세럼/앰플'
    
    elif cosme_df['소분류'][i] == 'Milk':
      cosme_df.loc[i, '중분류3'] = '로션'
  
    elif cond_extra[i] == True:
      cosme_df.loc[i, '중분류3'] = '기타'
#---------------------------------------------------------#
    elif cond_sun[i] == True:
      cosme_df.loc[i, '중분류3'] = '선케어'
#---------------------------------------------------------#    
    elif cond_found[i] == True:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'

    elif cond_blusher[i] == True:
      cosme_df.loc[i, '중분류3'] = '블러셔/쉐이딩/하이라이터'
    
    elif cond_loose[i] == True:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'
    
    elif cond_base[i] == True:
      cosme_df.loc[i, '중분류3'] = '프라이머/베이스'
    
    elif cosme_df['소분류'][i] == 'Concealer':
      cosme_df.loc[i, '중분류3'] = '컨실러'
#---------------------------------------------------------#    
    elif cond_balm[i] == True:
      cosme_df.loc[i, '중분류3'] = '립밤'
      
    elif cond_liner[i] == True:
      cosme_df.loc[i, '중분류3'] = '립라이너'    
#---------------------------------------------------------#   
    elif cond_shadow[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이섀도우'
      
    elif cond_eyeline[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이라이너'  

    elif cond_brow[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이브로우'  
    
    elif cond_masc[i] == True:
      cosme_df.loc[i, '중분류3'] = '마스카라'

    elif cond_eyelash[i] == True:
      cosme_df.loc[i, '중분류3'] = '속눈썹영양제'
    
    else:
      cosme_df.loc[i, '중분류3'] = '분류없음'

In [ ]:
# 고유값 수 확인인
print(cosme_df['중분류3'].unique())
print(cosme_df['중분류3'].nunique())
cosme_df.loc[:10, ['소분류', '중분류3']]

['마스크팩' '컨실러' '립스틱' '파운데이션/쿠션/BB/CC' '아이브로우' '에센스/세럼/앰플' '토너/토너패드/스킨'
 '파우더/팩트/픽서' '크림' '선케어' '아이섀도우' '마스카라' '립밤' '클렌징' '기타' '립글로스' '아이라이너'
 '프라이머/베이스' '블러셔/쉐이딩/하이라이터' '립틴트' '로션' '속눈썹영양제' '립라이너']
23


,소분류,중분류3
0,Sheet mask pack,마스크팩
1,Concealer,컨실러
2,lipstick,립스틱
3,Liquid foundation,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩
5,Brow mascara,아이브로우
6,Beauty fluid,에센스/세럼/앰플
7,Lotion,토너/토너패드/스킨
8,Sheet mask pack,마스크팩
9,Sheet mask pack,마스크팩


In [ ]:
# 중분류2 생성
cosme_df['중분류2'] = cosme_df['중분류3'].copy()
cosme_df['중분류2'].head(10)

0              마스크팩
1               컨실러
2               립스틱
3    파운데이션/쿠션/BB/CC
4              마스크팩
5             아이브로우
6         에센스/세럼/앰플
7        토너/토너패드/스킨
8              마스크팩
9              마스크팩
Name: 중분류2, dtype: object

In [ ]:
# 중분류2 수정
# 중분류3('립틴트', '립라이너', '립글로스', '립밤', '립스틱스틱') => 중분류2(립메이크업) 으로 일괄 변경 

lip_list = ['립틴트', '립라이너', '립글로스', '립밤', '립스틱']
lips = '|'.join(lip_list)
cond_lips = cosme_df['중분류3'].str.contains(lips)

for i in range(len(cosme_df)):
  if cond_lips[i] == True:
    cosme_df.loc[i, '중분류2'] = '립메이크업'

In [ ]:
print(cosme_df['중분류2'].unique())
print(cosme_df['중분류2'].nunique())
cosme_df.loc[:, ['소분류', '중분류2', '중분류3']]

['마스크팩' '컨실러' '립메이크업' '파운데이션/쿠션/BB/CC' '아이브로우' '에센스/세럼/앰플' '토너/토너패드/스킨'
 '파우더/팩트/픽서' '크림' '선케어' '아이섀도우' '마스카라' '클렌징' '기타' '아이라이너' '프라이머/베이스'
 '블러셔/쉐이딩/하이라이터' '로션' '속눈썹영양제']
19


,소분류,중분류2,중분류3
0,Sheet mask pack,마스크팩,마스크팩
1,Concealer,컨실러,컨실러
2,lipstick,립메이크업,립스틱
3,Liquid foundation,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩,마스크팩
...,...,...,...
2576,Lip care and lip balm,립메이크업,립밤
2577,Gel and cream eyeshadow,아이섀도우,아이섀도우
2578,Beauty fluid,에센스/세럼/앰플,에센스/세럼/앰플
2579,Pencil eyeliner,아이라이너,아이라이너


In [ ]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = cosme_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = cosme_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = cosme_df['중분류2'].str.contains(eye)

for i in range(len(cosme_df)):
  if cond_skincare[i] == True:
    cosme_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    cosme_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    cosme_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if cosme_df['중분류2'][i] == '클렌징':
      cosme_df.loc[i, '중분류1'] = '클렌징'

    elif cosme_df['중분류2'][i] == '마스크팩':
      cosme_df.loc[i, '중분류1'] = '마스크팩'
      
    elif cosme_df['중분류2'][i] == '기타':
      cosme_df.loc[i, '중분류1'] = '기타'
      
    elif cosme_df['중분류2'][i] == '선케어':
      cosme_df.loc[i, '중분류1'] = '선케어'
    
    elif cosme_df['중분류2'][i] == '립메이크업':
      cosme_df.loc[i, '중분류1'] = '립메이크업'

In [ ]:
print(cosme_df['중분류1'].unique())
print(cosme_df['중분류1'].nunique())
cosme_df.loc[:, ['소분류', '중분류1', '중분류2', '중분류3']]

['마스크팩' '베이스메이크업' '립메이크업' '아이메이크업' '스킨케어' '선케어' '클렌징' '기타']
8


,소분류,중분류1,중분류2,중분류3
0,Sheet mask pack,마스크팩,마스크팩,마스크팩
1,Concealer,베이스메이크업,컨실러,컨실러
2,lipstick,립메이크업,립메이크업,립스틱
3,Liquid foundation,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩,마스크팩,마스크팩
...,...,...,...,...
2576,Lip care and lip balm,립메이크업,립메이크업,립밤
2577,Gel and cream eyeshadow,아이메이크업,아이섀도우,아이섀도우
2578,Beauty fluid,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플
2579,Pencil eyeliner,아이메이크업,아이라이너,아이라이너


In [ ]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = cosme_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = cosme_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = cosme_df['중분류1'].str.contains(colorclass)

for i in range(len(cosme_df)):
  if cond_skinclass[i] == True:
    cosme_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    cosme_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    cosme_df.loc[i, '대분류'] = '색조메이크업'

In [ ]:
# 대분류 고유값 확인 
print(cosme_df['대분류'].unique())
print(cosme_df['대분류'].nunique())
cosme_df.loc[:, ['대분류', '소분류', '중분류1', '중분류2', '중분류3']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,소분류,중분류1,중분류2,중분류3
0,스킨케어,Sheet mask pack,마스크팩,마스크팩,마스크팩
1,베이스메이크업,Concealer,베이스메이크업,컨실러,컨실러
2,색조메이크업,lipstick,립메이크업,립메이크업,립스틱
3,베이스메이크업,Liquid foundation,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,스킨케어,Sheet mask pack,마스크팩,마스크팩,마스크팩
...,...,...,...,...,...
2576,색조메이크업,Lip care and lip balm,립메이크업,립메이크업,립밤
2577,색조메이크업,Gel and cream eyeshadow,아이메이크업,아이섀도우,아이섀도우
2578,스킨케어,Beauty fluid,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플
2579,색조메이크업,Pencil eyeliner,아이메이크업,아이라이너,아이라이너


### 데이터셋 정리

In [ ]:
# 기존 카테고리 지우기
cosme_df.drop(columns={'제품ID', '소분류'}, axis=1, inplace=True)
cosme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   순위      2581 non-null   int64  
 2   브랜드     2581 non-null   object 
 3   제품명     2581 non-null   object 
 4   평점      2581 non-null   float64
 5   후기 수    2581 non-null   int64  
 6   가격      2581 non-null   int64  
 7   중분류1    2581 non-null   object 
 8   중분류3    2581 non-null   object 
 9   중분류2    2581 non-null   object 
 10  대분류     2581 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 221.9+ KB


In [ ]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['순위', '대분류', '중분류1', '중분류2', '중분류3']

before_a = cosme_df.drop(columns = {'순위', '대분류', '중분류1', '중분류2', '중분류3'}).columns[:3].to_list()
after_a = cosme_df.drop(columns = {'순위', '대분류', '중분류1', '중분류2', '중분류3'}).columns[3:].to_list()

reorder = before_a + a + after_a
cosme_df = cosme_df[reorder]

cosme_df.columns

Index(['플랫폼', '브랜드', '제품명', '순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수',
       '가격'],
      dtype='object')

In [ ]:
# 순위 재정립 : 인덱스 + 1

for i in range(0, len(cosme_df)):
  cosme_df['순위'][i] = i+1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i] = i+1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i] = i+1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i

In [ ]:
# 최종 데이터셋 확인
print(cosme_df.info())
cosme_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   브랜드     2581 non-null   object 
 2   제품명     2581 non-null   object 
 3   순위      2581 non-null   int64  
 4   대분류     2581 non-null   object 
 5   중분류1    2581 non-null   object 
 6   중분류2    2581 non-null   object 
 7   중분류3    2581 non-null   object 
 8   평점      2581 non-null   float64
 9   후기 수    2581 non-null   int64  
 10  가격      2581 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 221.9+ KB
None


,플랫폼,브랜드,제품명,순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,코스메,브이티코스메틱,CICA Daily Soothing Mask,1,스킨케어,마스크팩,마스크팩,마스크팩,5.2,10034,2420
1,코스메,더샘,CP Chip Concealer,2,베이스메이크업,베이스메이크업,컨실러,컨실러,4.6,8804,858
2,코스메,롬앤,Juicy Lasting Tint,3,색조메이크업,립메이크업,립메이크업,립스틱,4.9,6709,1320
3,코스메,티르티르,MASK FIT RED CUSHION,4,베이스메이크업,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC,4.8,6530,2970
4,코스메,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,5,스킨케어,마스크팩,마스크팩,마스크팩,5.2,6457,1090


In [ ]:
# 최종 데이터셋 저장
# cosme_df.to_csv('cosme_preprocessing.csv', index=False, encoding="utf-8-sig")

# 리뷰 크롤링